In [40]:
# Load in our libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib as mpl
%matplotlib inline
mpl.style.use('seaborn')

import warnings
warnings.filterwarnings('ignore')

import time
import datetime as dt

from sklearn import preprocessing
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier,\
                              GradientBoostingClassifier)
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

import math
import statistics

In [2]:
from data_processing import *

# Stable data 

In [3]:
data,ETFtable,features = get_dflist()

# Target data 

In [4]:
def get_targetdata(ETFtable,code):
    ETF_price = extract_target_ETF(ETFtable,code)
    ETF_rate = get_change_rate(ETF_price)
    mrf = pd.Series(get_mean_rf(ETF_price ,'close',20,5),index=ETF_price.index)
    return [ETF_price,ETF_rate,mrf]

In [5]:
def get_dataframe(df_list,df_r,target,interval,window,ca=4,day = 1):
    df_list_rate = []
    df_rf = get_rise_fall(target,interval).shift(-1*day+1)
    
    #concat the df in the df_list
    for index,df in enumerate(df_list):
        if(df_r[index]):
            df_rate = get_change_rate(df,window).iloc[:,df.shape[1]:]
        else:
            df_rate = get_backday(df,window).iloc[:,df.shape[1]:]
            
        df_av  = get_av_ca(stock=df_rate,rf_df=df_rf,ca=ca,window = window).iloc[:,:-3]
        df_colname = list(map(lambda x:  df_name[index]+'_'+x,df_av.columns))
        df_dum = pd.get_dummies(df_av,prefix=df_colname,columns=df_av.columns)
        df_list_rate.append(df_dum)
        
    df_all = pd.concat(df_list_rate,axis =1)
    df_all['mean_rise_fall'] = mrf
    
    #create null columns for the null data
    df_null = df_all.copy()
    for columnname in df_all.columns:
        df_null[columnname+'isnull'] = list(map(lambda x: int(x),df_all.isna()[columnname]))
    df_null.drop(df_all.columns,axis = 1,inplace = True)

    df_total = pd.concat([df_all,df_null],axis = 1)
    df_total = pd.concat([df_total,df_rf.loc[df_total.index]],axis = 1).loc[TXF.index[25]:TXF.index[-1],:].fillna(0)
    
    #get target rise fall data
    Y = df_list[0].copy()
    Y['trf'] = np.sign((df_list[0]-df_list[0].shift(1))['close'].values)
    Y = Y.shift(-1*day+1)
    trf = Y.loc[df_total.index]['trf']

    return [df_total ,trf]


In [6]:
def get_dataframe_drop(df_list,df_r,target,interval,window,ca=4,day = 1):
    df_list_rate = []
    df_rf = get_rise_fall(target,interval).shift(-1*day+1)
    
    #concat the df in the df_list
    for index,df in enumerate(df_list):
        if(df_r[index]):
            df_rate = get_change_rate(df,window).iloc[:,df.shape[1]:]
        else:
            df_rate = get_backday(df,window).iloc[:,df.shape[1]:]
            
        df_av  = get_av_ca(stock=df_rate,rf_df=df_rf,ca=ca,window = window).iloc[:,:-3]
        df_colname = list(map(lambda x:  df_name[index]+'_'+x,df_av.columns))
        df_dum = pd.get_dummies(df_av,prefix=df_colname,columns=df_av.columns)
        df_list_rate.append(df_dum)
        
    df_all = pd.concat(df_list_rate,axis =1)
    df_all['mean_rise_fall'] = mrf
    
    df_total = df_all.dropna(axis = 0,how = 'any')
    df_total = pd.concat([df_total,df_rf.loc[df_total.index]],axis = 1).dropna(axis = 0,how = 'any')

    #get target rise fall data
    Y = df_list[0].copy()
    Y['trf'] = np.sign((df_list[0]-df_list[0].shift(1))['close'].values)
    Y = Y.shift(-1*day+1)
    trf = Y.loc[df_total.index]['trf']

    return [df_total,trf]


In [7]:
def get_dataframe_rf(df_list,df_r,target,interval,window,ca=4,day = 1):
    df_list_rate = []
    df_rf = get_rise_fall(target,interval).shift(-1*day+1)
    
    #concat the df in the df_list
    for index,df in enumerate(df_list):
        if(df_r[index]):
            df_rate = get_change_rate(df,window).iloc[:,df.shape[1]:]
        else:
            df_rate = get_backday(df,window).iloc[:,df.shape[1]:]
            
        df_av  = get_av_ca(stock=df_rate,rf_df=df_rf,ca=ca,window = window).iloc[:,:-3]
        df_colname = list(map(lambda x:  df_name[index]+'_'+x,df_av.columns))
        df_dum = pd.get_dummies(df_av,prefix=df_colname,columns=df_av.columns)
        df_list_rate.append(df_dum)
        
    df_all = pd.concat(df_list_rate,axis =1)
    df_all['mean_rise_fall'] = mrf
    
    df_total = df_all.dropna(axis = 0,how = 'any')
    
    #split rise and fall
    dicu = {}
    dicd = {}
    for index, col in enumerate(df_total.columns[:-1]):
        dicu[col] = col+'_r'
        dicd[col] = col+'_f'

    up = df_total[df_total['mean_rise_fall']==1].rename(columns = dicu).iloc[:,:-1]
    dn = df_total[df_total['mean_rise_fall']==-1].rename(columns = dicd).iloc[:,:-1]
    df_total_ud = pd.concat([up,dn],axis = 1).sort_index().fillna(0)
    
    df_total_ud = pd.concat([df_total_ud,df_rf.loc[df_total_ud.index]],axis = 1).dropna(axis = 0,how = 'any')
    
    #get target rise fall data
    Y = df_list[0].copy()
    Y['trf'] = np.sign((df_list[0]-df_list[0].shift(1))['close'].values)
    Y = Y.shift(-1*day+1)
    trf = Y.loc[df_total_ud.index]['trf']

    return [df_total_ud,trf]

# Model

In [46]:
models  =  {
#     'DecisionTreeClassifier' : DecisionTreeClassifier(random_state = 0),
#          'RandomForestClassifier': RandomForestClassifier(random_state = 0),
#     'ExtraTreesClassifier': ExtraTreesClassifier(random_state = 0),
    'XGBClassifier' : XGBClassifier(random_state=0),
#     'AdaBoostClassifier': AdaBoostClassifier(base_estimator = DecisionTreeClassifier(),\
#                                              n_estimators = 10,random_state = 0),
#     'GradientBoostingClassifier': GradientBoostingClassifier(random_state = 0),
#     'SVC': SVC(probability=True,random_state = 0),
}
    
model_grid_params = {
#         'DecisionTreeClassifier':{'max_depth': [None,1,5,10],'min_samples_leaf': [1,2,5,10]},
#     'RandomForestClassifier': {'max_features':[None],'n_estimators':[10,50],'max_depth':[2,10],\
#                                'min_samples_split':[2],#'criterion':['entropy'],\
#                                'min_samples_leaf':[3]},
    'XGBClassifier' : {'n_estimators':[10,20,100],'max_depth':[2,5],},
#     'ExtraTreesClassifier': {'max_features':[None],'n_estimators':[10,50],'max_depth':[2,10],\
#                              'min_samples_split':[2],'criterion':['entropy'],\
#                              'min_samples_leaf':[3]},
#     'AdaBoostClassifier': {"base_estimator__criterion" : ["entropy"],\
#                            "base_estimator__max_depth": [None],\
#                            "base_estimator__min_samples_leaf" : [3,10],\
#                            "base_estimator__min_samples_split" : [2,10],\
#                            "base_estimator__max_features" : [None]},
#     'GradientBoostingClassifier': {'max_features':[None],'n_estimators':[10],'max_depth':[10],\
#                                    'min_samples_split':[2],'min_samples_leaf':[3],\
#                                    'learning_rate':[0.1],'loss':['deviance']},
#     'SVC': [{'kernel':['rbf'],'gamma':[1e-1],'C':[1]},\
#             {'kernel':['linear'],'C':[1,10]}]
}

In [9]:
class Model_Selection3:
    
    def __init__(self,models,model_grid_params,stock_5,rf_5,latest_day,pred_day,day,window = 1):
        
        self.week_score = []
        self.models = models
        self.model_grid = model_grid_params
        self.stock_5 = stock_5
        self.rf_5 = rf_5
        self.latest_day= latest_day
        self.pred_day = pred_day
        self.day = day
        self.keys = models.keys()
        self.best_score = {}
        self.grid = {}
        
        self.predict_values = {}
#         self.predict_values = {}
        self.cv_acc = {}
        self.acc = {}
        self.fscore = {}
        self.true_values = {}
        
        self.predict_values_day = {}
        self.cv_acc_day = {}
        self.acc_day = {}
        self.fscore_day = {}
        self.true_values_day = {}
        self.summary_day = []
        
    def Grid_fit(self,X_train,y_train,cv = 5,scoring = 'accuracy'):
        
        for key in self.keys:
            print ("Running GridSearchCV for %s" %(key))
            model = self.models[key]
            model_grid = self.model_grid[key]
            Grid = GridSearchCV(model, model_grid, cv = cv, scoring = scoring)
            Grid.fit(X_train,y_train) 
            self.grid[key] = Grid
            print (Grid.best_params_)
            print ('CV Best Score = %s'%(Grid.best_score_))
            self.cv_acc[key].append(Grid.best_score_)  
    
    def model_fit(self,X_train, y_train, X_test, y_test):
        
        for key in self.keys:
            print ("Running training & testing for %s." %(key))
            model = self.models[key]
            model.set_params(**self.grid[key].best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            #print 'Prediction latest 15 second = %s'%(predictions)
            self.predict_values[key].append(predictions.tolist())
            self.true_values[key].append(y_test.tolist())
            acc = metrics.accuracy_score(y_test,predictions)
            f_score = metrics.f1_score(y_test,predictions)
            print ('Accuracy = %s'%(acc))
            self.acc[key].append(acc)
            self.fscore[key].append(f_score)

    def pipline(self):
        
        self.set_list_day() # store day values
        for day in range(0,self.day,1):
            self.set_list() # store values
            print ('Day = %s'%(day+1))
            check1 = self.stock_5[-1].shape[0]-1000-self.pred_day-self.latest_day[day]
            check2 = self.stock_5[-1].shape[0]-100-self.pred_day-self.latest_day[day]-4
            if (check1 > 0):
                for k in range(0,1000,self.latest_day[day]):
                    i = k + self.stock_5[day].shape[0]-1000-self.pred_day-self.latest_day[day]
                    print ('--------------------Rolling Window Time = %s--------------------'%(k/self.latest_day[day]))
                    # Train data
                    X_train = self.stock_5[day][i:i+self.latest_day[day]]
                    data_train = self.rf_5[day][i:i+self.latest_day[day]]

                    train_rise = data_train[:,-3]
                    train_fall = data_train[:,-2]
                    train_noise = data_train[:,-1]
                    y_train = train_rise

                    # Test data
                    print(i)
                    X_test = self.stock_5[day][i + self.latest_day[day]:i + self.latest_day[day] + self.pred_day]
                    data_test = self.rf_5[day][i + self.latest_day[day]:i + self.latest_day[day] + self.pred_day]
                    test_rise = data_test[:,-3]
                    test_fall = data_test[:,-2]
                    test_noise = data_test[:,-1]
                    y_test = test_rise

                    #start = time.time()
                    self.Grid_fit(X_train, y_train, cv = 2, scoring = 'accuracy')
                    self.model_fit(X_train, y_train,X_test,y_test)
                    #end = time.time()
                    #print 'Total Time = %s'%(end - start)
            elif(check2 > 0):
                for k in range(0,100,self.latest_day[day]):
                    i = k + self.stock_5[day].shape[0]-100-self.pred_day-self.latest_day[day]
                    print ('--------------------Rolling Window Time = %s--------------------'%(k/latest_day[day]))
                    # Train data
                    X_train = self.stock_5[day][i:i+self.latest_day[day]]
                    data_train = self.rf_5[day][i:i+self.latest_day[day]]

                    train_rise = data_train[:,-3]
                    train_fall = data_train[:,-2]
                    train_noise = data_train[:,-1]
                    y_train = train_rise

                    # Test data
                    X_test = self.stock_5[day][i + self.latest_day[day]:i + self.latest_day[day] + self.pred_day]
                    data_test = self.rf_5[day][i + self.latest_day[day]:i + self.latest_day[day] + self.pred_day]
                    test_rise = data_test[:,-3]
                    test_fall = data_test[:,-2]
                    test_noise = data_test[:,-1]
                    y_test = test_rise

                    #start = time.time()
                    self.Grid_fit(X_train, y_train, cv = 2, scoring = 'accuracy')
                    self.model_fit(X_train, y_train,X_test,y_test)
                    #end = time.time()
                    #print 'Total Time = %s'%(end - start)
            else:
                
                i = 4
                print ('--------------------Rolling Window Time = %s--------------------'%(i/pred_day))
                # Train data
                X_train = self.stock_5[day][i:-1*self.pred_day]
                data_train = self.rf_5[day][i:-1*self.pred_day]

                train_rise = data_train[:,-3]
                train_fall = data_train[:,-2]
                train_noise = data_train[:,-1]
                y_train = train_rise

                # Test data
                X_test = self.stock_5[day][-1*self.pred_day:]
                data_test = self.rf_5[day][-1*self.pred_day:]
                test_rise = data_test[:,-3]
                test_fall = data_test[:,-2]
                test_noise = data_test[:,-1]
                y_test = test_rise

                #start = time.time()
                self.Grid_fit(X_train, y_train, cv = 2, scoring = 'accuracy')
                self.model_fit(X_train, y_train,X_test,y_test)
                #end = time.time()
                #print 'Total Time = %s'%(end - start)
                
            for key in self.keys:
                
                self.cv_acc_day[key].append(self.cv_acc[key])
                self.acc_day[key].append(self.acc[key])
                self.fscore_day[key].append(self.fscore[key])
                self.true_values_day[key].append(self.true_values[key])
                self.predict_values_day[key].append(self.predict_values[key])
            if (check2 > 0):
                self.summary_day.append(self.score_summary(sort_by = 'Accuracy_mean'))
            else:
                self.summary_day.append(self.score_summary_1cv(sort_by = 'Accuracy_mean'))
            
    def model_pre(self,X_train,y_train,X_test):
        for key in self.keys:
            print ("Running training & testing for %s." %(key))
            model = self.models[key]
            model.set_params(**self.grid[key].best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            #print 'Prediction latest 15 second = %s'%(predictions)
            self.predict_values[key].append(predictions.tolist())
            
        
    def pipline_pre(self):
        self.set_list_day() # store day values
        for day in range(0,self.day,1):
            self.set_list() # store values
            print ('Day = %s'%(day+1))
            
            # Train data
            X_train = self.stock_5[day][-1*self.latest_day[day]:]
            data_train = self.rf_5[day][-1*self.latest_day[day]:]
            
            train_rise = data_train[:,-3]
            train_fall = data_train[:,-2]
            train_noise = data_train[:,-1]
            y_train = train_rise
            
            #test 有BUG
            X_test = self.stock_5[day][-1*window:,:]
#             X_test = self.true_stock[-1:,:]
            
            #start = time.time()
            self.Grid_fit(X_train, y_train, cv = 2, scoring = 'accuracy')
            self.model_pre(X_train, y_train , X_test)
            #end = time.time()
            #print 'Total Time = %s'%(end - start)
                
            for key in self.keys:
                
                self.cv_acc_day[key].append(self.cv_acc[key])
                self.acc_day[key].append(self.acc[key])
                self.fscore_day[key].append(self.fscore[key])
                self.true_values_day[key].append(self.true_values[key])
                self.predict_values_day[key].append(self.predict_values[key])
            
#             self.summary_day.append(self.score_summary(sort_by = 'Accuracy_mean'))
            
    
    def pipline_pre_4(self):
        pro = [0.1,0.15,0.2,0.25,0.3]
        
        for week in range(4):
            self.set_list_day() # store day values
            score_day  = 0
            for day in range(0,self.day,1):
                self.set_list() # store values
                print ('Day = %s'%(day+1))
                i = -20+5*week
                print ('--------------------Rolling Window Time = %s--------------------'%(week))
                # Train data
                X_train = self.stock_5[day][i-self.latest_day[day]:i]
                data_train = self.rf_5[day][i-self.latest_day[day]:i]

                train_rise = data_train[:,-3]
                train_fall = data_train[:,-2]
                train_noise = data_train[:,-1]
                y_train = train_rise

                # Test data
                if(week == 3 and day == 4):
                    X_test = self.stock_5[day][i+day:]
                    data_test = self.rf_5[day][i+day:]
                    test_rise = data_test[:,-3]
                    test_fall = data_test[:,-2]
                    test_noise = data_test[:,-1]
                    y_test = test_rise
                else:
                    X_test = self.stock_5[day][i+day:i+day+1]
                    data_test = self.rf_5[day][i+day:i+day+1]
                    test_rise = data_test[:,-3]
                    test_fall = data_test[:,-2]
                    test_noise = data_test[:,-1]
                    y_test = test_rise
                    
                #start = time.time()
                self.Grid_fit(X_train, y_train, cv = 2, scoring = 'accuracy')
                self.model_fit(X_train, y_train,X_test,y_test)
                #end = time.time()
                #print 'Total Time = %s'%(end - start)
                
                score_day += (self.acc['RandomForestClassifier'][0]*pro[day]*0.5)
                
                for key in self.keys:
                    self.cv_acc_day[key].append(self.cv_acc[key])
                    self.acc_day[key].append(self.acc[key])
                    self.fscore_day[key].append(self.fscore[key])
                    self.true_values_day[key].append(self.true_values[key])
                    self.predict_values_day[key].append(self.predict_values[key])
                
            self.week_score.append(score_day)
            
            
    def set_list(self):
        
        for key in self.keys:
            self.predict_values[key] = []
            self.cv_acc[key] = []
            self.acc[key] = []
            self.fscore[key] = []
            self.true_values[key] = []
            
    def set_list_day(self):
        
        for key in self.keys:
            self.predict_values_day[key] = []
            self.cv_acc_day[key] = []
            self.acc_day[key] = []
            self.fscore_day[key] = []
            self.true_values_day[key] = []
            
    def score_summary(self,sort_by):
        
        summary = pd.concat([pd.Series(list(self.acc.keys())),pd.Series(map(lambda x: sum(self.acc[x])/len(self.acc[x]), self.acc)),\
                             pd.Series(list(map(lambda x: statistics.stdev(self.acc[x]), self.acc))),\
                             pd.Series(list(map(lambda x: max(self.acc[x]), self.acc))),\
                             pd.Series(list(map(lambda x: min(self.acc[x]), self.acc))),\
                             pd.Series(list(map(lambda x: sum(self.fscore[x])/len(self.fscore[x]), self.fscore)))],axis=1)
        summary.columns = ['Estimator','Accuracy_mean','Accuracy_std','Accuracy_max','Accuracy_min','F_score']
        summary.index.rename('Ranking', inplace=True)
        return summary.sort_values(by = [sort_by], ascending=False)
    
    def score_summary_1cv(self,sort_by):
        
        summary = pd.concat([pd.Series(list(self.acc.keys())),pd.Series(map(lambda x: sum(self.acc[x])/len(self.acc[x]), self.acc)),\
                             
                             pd.Series(list(map(lambda x: max(self.acc[x]), self.acc))),\
                             pd.Series(list(map(lambda x: min(self.acc[x]), self.acc))),\
                             pd.Series(list(map(lambda x: sum(self.fscore[x])/len(self.fscore[x]), self.fscore)))],axis=1)
        summary.columns = ['Estimator','Accuracy_mean','Accuracy_max','Accuracy_min','F_score']
        summary.index.rename('Ranking', inplace=True)
        return summary.sort_values(by = [sort_by], ascending=False)
          
    def print_(self):

        print (self.predict_values)

In [10]:
class Model_Selection_split:
    
    def __init__(self,models,model_grid_params,stock_5,trf_5,latest_day,pred_day,day):
        
        self.models = models
        self.model_grid = model_grid_params
        self.stock_5 = stock_5
        self.trf_5 = trf_5
        self.latest_day= latest_day
        self.pred_day = pred_day
        self.day = day
        self.keys = models.keys()
    
        
        #rise
        self.best_score_r = {}
        self.grid_r = {}
        
        self.predict_values_r = {}
        self.predict_proba_r = {}
        self.cv_acc_r = {}
        self.acc_r = {}
        self.fscore_r = {}
        self.true_values_r = {}
        
        self.predict_values_day_r = {}
        self.predict_proba_day_r = {}
        self.cv_acc_day_r = {}
        self.acc_day_r = {}
        self.fscore_day_r = {}
        self.true_values_day_r = {}
        self.summary_day_r = []
        
        #fall
        self.best_score_f = {}
        self.grid_f = {}
        
        self.predict_values_f = {}
        self.predict_proba_f = {}
        self.cv_acc_f = {}
        self.acc_f = {}
        self.fscore_f = {}
        self.true_values_f = {}
        
        self.predict_values_day_f = {}
        self.predict_proba_day_f = {}
        self.cv_acc_day_f = {}
        self.acc_day_f = {}
        self.fscore_day_f = {}
        self.true_values_day_f = {}
        self.summary_day_f = []
        
        #total
        self.acc ={}
        self.fscore = {}
        
        self.acc_day ={}
        self.fscore_day = {}
        
        self.summary_day = []
        
    def Grid_fit_r(self,X_train,y_train,cv = 5,scoring = 'accuracy'):
        
        for key in self.keys:
            print ("Running GridSearchCV for %s" %(key))
            model = self.models[key]
            model_grid = self.model_grid[key]
            Grid = GridSearchCV(model, model_grid, cv = cv, scoring = scoring)
            Grid.fit(X_train,y_train) 
            self.grid_r[key] = Grid
            print (Grid.best_params_)
            print ('CV Best Score = %s'%(Grid.best_score_))
            self.cv_acc_r[key].append(Grid.best_score_)  
            
    def Grid_fit_f(self,X_train,y_train,cv = 5,scoring = 'accuracy'):
        
        for key in self.keys:
            print ("Running GridSearchCV for %s" %(key))
            model = self.models[key]
            model_grid = self.model_grid[key]
            Grid = GridSearchCV(model, model_grid, cv = cv, scoring = scoring)
            Grid.fit(X_train,y_train) 
            self.grid_f[key] = Grid
            print (Grid.best_params_)
            print ('CV Best Score = %s'%(Grid.best_score_))
            self.cv_acc_f[key].append(Grid.best_score_) 
    
    def model_fit_r(self,X_train, y_train, X_test, y_test):
        
        for key in self.keys:
            print ("Running training & testing for %s." %(key))
            model = self.models[key]
            model.set_params(**self.grid_r[key].best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            proba = model.predict_proba(X_test)[:,0]
            #print 'Prediction latest 15 second = %s'%(predictions)
            self.predict_values_r[key].append(predictions.tolist())
            self.predict_proba_r[key].append(proba.tolist())
            
            self.true_values_r[key].append(y_test.tolist())
            acc = metrics.accuracy_score(y_test,predictions)
            f_score = metrics.f1_score(y_test,predictions)
            print ('Accuracy = %s'%(acc))
            self.acc_r[key].append(acc)
            self.fscore_r[key].append(f_score)
        
    def model_fit_f(self,X_train, y_train, X_test, y_test):
        
        for key in self.keys:
            print ("Running training & testing for %s." %(key))
            model = self.models[key]
            model.set_params(**self.grid_f[key].best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            proba = model.predict_proba(X_test)[:,0]
            #print 'Prediction latest 15 second = %s'%(predictions)
            self.predict_values_f[key].append(predictions.tolist())
            self.predict_proba_f[key].append(proba.tolist())
            self.true_values_f[key].append(y_test.tolist())
            acc = metrics.accuracy_score(y_test,predictions)
            f_score = metrics.f1_score(y_test,predictions)
            print ('Accuracy = %s'%(acc))
            self.acc_f[key].append(acc)
            self.fscore_f[key].append(f_score)
            
    def model_fit(self,v1,v2,v3,v4,v5,y_test):
        
        for key in self.keys:
            r1_ = np.array(v1[key][0])
            r2_ = np.array(v2[key][0])
            f1_ = np.array(v3[key][0])
            f2_ = np.array(v4[key][0])
            noi_ = np.array(v5[key][0])
            
            v = []
            for day in range(r1_.shape[0]):
                if(r1_[day] == 1):
                    v.append(1)
                elif (r2_[day] == 1):
                    v.append(-1)
                elif(r2_[day] == 1):
                    v.append(1)
                elif(f2_[day] == 1):
                    v.append(-1)
                elif(noi_[day] == 1):
                    v.append(1)
                else:
                    v.append(-1)
                    
            acc = metrics.accuracy_score(y_test,v)
#             f_score = metrics.f1_score(y_test,predictions)
            print ('Accuracy = %s'%(acc))
            self.acc[key].append(acc)
#             self.fscore[key].append(f_score)
            
    def pipline(self):
        
        self.set_list_day() # store day values
        for day in range(0,self.day,1):
            self.set_list() # store values
            print ('Day = %s'%(day+1))
            for i in range(0,500,self.pred_day):#9000-self.latest_day-600,self.pred_day):
                
                print ('--------------------Rolling Window Time = %s--------------------'%(i/pred_day))
                
                # Train data
                data_train = self.stock_5[day][i:i+self.latest_day]
                X_train = data_train[:,:-5]
                train_rise1 = data_train[:,-5]
                train_rise2 = data_train[:,-4]
                train_noise = data_train[:,-3]
                train_fall2 = data_train[:,-2]
                train_fall1= data_train[:,-1]
                
                # Test data
                data_test = self.stock_5[day][i + self.latest_day:i + self.latest_day + self.pred_day]
                X_test = data_test[:,:-5]
                test_rise1 = data_test[:,-5]
                test_rise2 = data_test[:,-4]
                test_noise = data_test[:,-3]
                test_fall2 = data_test[:,-2]
                test_fall1= data_test[:,-1]
                
                y_test = self.trf_5[day][i + self.latest_day:i + self.latest_day + self.pred_day] 
                
                #start = time.time()
                self.Grid_fit_r(X_train, train_rise1, cv = 2, scoring = 'accuracy')
                self.model_fit_r(X_train, train_rise1,X_test,test_rise1)
                r1 = self.predict_values_r.copy()
                self.set_list_r()
                
                self.Grid_fit_r(X_train, train_rise2, cv = 2, scoring = 'accuracy')
                self.model_fit_r(X_train, train_rise2,X_test,test_rise2)
                r2 = self.predict_values_r.copy()
                self.set_list_r()
                
                self.Grid_fit_f(X_train, train_fall1, cv = 2, scoring = 'accuracy')
                self.model_fit_f(X_train, train_fall1,X_test,test_fall1)
                f1 = self.predict_values_f.copy()
                self.set_list_f()
                
                self.Grid_fit_f(X_train, train_fall2, cv = 2, scoring = 'accuracy')
                self.model_fit_f(X_train, train_fall2,X_test,test_fall2)
                f2 = self.predict_values_f.copy()
                self.set_list_f()
                
                self.Grid_fit_r(X_train, train_noise, cv = 2, scoring = 'accuracy')
                self.model_fit_r(X_train, train_noise,X_test,test_noise)
                noi = self.predict_values_r.copy()
                self.set_list_r()
                
                self.model_fit(r1,r2,f1,f2,noi,y_test)
                #end = time.time()
                #print 'Total Time = %s'%(end - start)
                
            for key in self.keys:
                
                self.cv_acc_day_r[key].append(self.cv_acc_r[key])
                self.acc_day_r[key].append(self.acc_r[key])
                self.fscore_day_r[key].append(self.fscore_r[key])
                self.true_values_day_r[key].append(self.true_values_r[key])
                self.predict_values_day_r[key].append(self.predict_values_r[key])
                self.predict_proba_day_r[key].append(self.predict_proba_r[key])
                
                self.cv_acc_day_f[key].append(self.cv_acc_f[key])
                self.acc_day_f[key].append(self.acc_f[key])
                self.fscore_day_f[key].append(self.fscore_f[key])
                self.true_values_day_f[key].append(self.true_values_f[key])
                self.predict_values_day_f[key].append(self.predict_values_f[key])
                self.predict_proba_day_f[key].append(self.predict_proba_f[key])
                
                self.acc_day[key].append(self.acc[key])
#                 self.fscore_day[key].append(self.f_score[key])
                
#             self.summary_day_r.append(self.score_summary_r(sort_by = 'Accuracy_mean'))
#             self.summary_day_f.append(self.score_summary_f(sort_by = 'Accuracy_mean'))
            self.summary_day.append(self.score_summary(sort_by = 'Accuracy_mean'))
    
    def set_list(self):
        
        for key in self.keys:
            self.predict_values_r[key] = []
            self.predict_proba_r[key] = []
            self.cv_acc_r[key] = []
            self.acc_r[key] = []
            self.fscore_r[key] = []
            self.true_values_r[key] = []
            
            self.predict_values_f[key] = []
            self.predict_proba_f[key] = []
            self.cv_acc_f[key] = []
            self.acc_f[key] = []
            self.fscore_f[key] = []
            self.true_values_f[key] = []
            
            self.acc[key] = []
            self.fscore[key] = []
        
    def set_list_r(self):
        for key in self.keys:
            self.predict_values_r[key] = []
            self.predict_proba_r[key] = []
            self.cv_acc_r[key] = []
            self.acc_r[key] = []
            self.fscore_r[key] = []
            self.true_values_r[key] = []
    
    def set_list_f(self):
        for key in self.keys:
            self.predict_values_f[key] = []
            self.predict_proba_f[key] = []
            self.cv_acc_f[key] = []
            self.acc_f[key] = []
            self.fscore_f[key] = []
            self.true_values_f[key] = []
            
    def set_list_day(self):
        
        for key in self.keys:
            self.predict_values_day_r[key] = []
            self.predict_proba_day_r[key] = []
            self.cv_acc_day_r[key] = []
            self.acc_day_r[key] = []
            self.fscore_day_r[key] = []
            self.true_values_day_r[key] = []
            
            self.predict_values_day_f[key] = []
            self.predict_proba_day_f[key] = []
            self.cv_acc_day_f[key] = []
            self.acc_day_f[key] = []
            self.fscore_day_f[key] = []
            self.true_values_day_f[key] = []
            
            self.acc_day[key] = []
            self.fscore_day[key] = []
            
    def score_summary_r(self,sort_by):
        
        summary_r = pd.concat([pd.Series(list(self.acc_r.keys())),pd.Series(map(lambda x: sum(self.acc_r[x])/len(self.acc_r[x]), self.acc_r)),\
                             pd.Series(list(map(lambda x: statistics.stdev(self.acc_r[x]), self.acc_r))),\
                             pd.Series(list(map(lambda x: max(self.acc_r[x]), self.acc_r))),\
                             pd.Series(list(map(lambda x: min(self.acc_r[x]), self.acc_r))),\
                             pd.Series(list(map(lambda x: sum(self.fscore_r[x])/len(self.fscore_r[x]), self.fscore_r)))],axis=1)
        summary_r.columns = ['Estimator','Accuracy_mean','Accuracy_std','Accuracy_max','Accuracy_min','F_score']
        summary_r.index.rename('Ranking', inplace=True)
        return summary_r.sort_values(by = [sort_by], ascending=False)
    
    def score_summary_f(self,sort_by):
        
        summary_f = pd.concat([pd.Series(list(self.acc_f.keys())),pd.Series(map(lambda x: sum(self.acc_f[x])/len(self.acc_f[x]), self.acc_f)),\
                             pd.Series(list(map(lambda x: statistics.stdev(self.acc_f[x]), self.acc_f))),\
                             pd.Series(list(map(lambda x: max(self.acc_f[x]), self.acc_f))),\
                             pd.Series(list(map(lambda x: min(self.acc_f[x]), self.acc_f))),\
                             pd.Series(list(map(lambda x: sum(self.fscore_f[x])/len(self.fscore_f[x]), self.fscore_f)))],axis=1)
        summary_f.columns = ['Estimator','Accuracy_mean','Accuracy_std','Accuracy_max','Accuracy_min','F_score']
        summary_f.index.rename('Ranking', inplace=True)
        return summary_f.sort_values(by = [sort_by], ascending=False)
    
    def score_summary(self,sort_by):         
        summary = pd.concat([pd.Series(list(self.acc.keys())),pd.Series(map(lambda x: sum(self.acc[x])/len(self.acc[x]), self.acc)),\
                             pd.Series(list(map(lambda x: statistics.stdev(self.acc[x]), self.acc))),\
                             pd.Series(list(map(lambda x: max(self.acc[x]), self.acc))),\
                             pd.Series(list(map(lambda x: min(self.acc[x]), self.acc)))],axis = 1)
#                              pd.Series(list(map(lambda x: sum(self.fscore[x])/len(self.fscore[x]), self.fscore)))],axis=1)
                             
        summary.columns = ['Estimator','Accuracy_mean','Accuracy_std','Accuracy_max','Accuracy_min']
        summary.index.rename('Ranking', inplace=True)
        return summary.sort_values(by = [sort_by], ascending=False)
    
    def print_(self):

        print (self.predict_values)

In [11]:
class Model_Selection2:
    
    def __init__(self,models,model_grid_params,stock_5,rf_5,latest_day,pred_day,day,window = 1):
        
        self.models = models
        self.model_grid = model_grid_params
        self.stock_5 = stock_5
        self.rf_5 = rf_5
        self.latest_day= latest_day
        self.pred_day = pred_day
        self.day = day
        self.keys = models.keys()
        self.best_score = {}
        self.grid = {}
        
        self.predict_values = {}
#         self.predict_values = {}
        self.cv_acc = {}
        self.acc = {}
        self.fscore = {}
        self.true_values = {}
        
        self.predict_values_day = {}
        self.cv_acc_day = {}
        self.acc_day = {}
        self.fscore_day = {}
        self.true_values_day = {}
        self.summary_day = []
        
    def Grid_fit(self,X_train,y_train,cv = 5,scoring = 'accuracy'):
        
        for key in self.keys:
            print ("Running GridSearchCV for %s" %(key))
            model = self.models[key]
            model_grid = self.model_grid[key]
            Grid = GridSearchCV(model, model_grid, cv = cv, scoring = scoring)
            Grid.fit(X_train,y_train) 
            self.grid[key] = Grid
            print (Grid.best_params_)
            print ('CV Best Score = %s'%(Grid.best_score_))
            self.cv_acc[key].append(Grid.best_score_)  
    
    def model_fit(self,X_train, y_train, X_test, y_test):
        
        for key in self.keys:
            print ("Running training & testing for %s." %(key))
            model = self.models[key]
            model.set_params(**self.grid[key].best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            #print 'Prediction latest 15 second = %s'%(predictions)
            self.predict_values[key].append(predictions.tolist())
            self.true_values[key].append(y_test.tolist())
            acc = metrics.accuracy_score(y_test,predictions)
            f_score = metrics.f1_score(y_test,predictions)
            print ('Accuracy = %s'%(acc))
            self.acc[key].append(acc)
            self.fscore[key].append(f_score)

    def pipline(self):
        
        self.set_list_day() # store day values
        for day in range(0,self.day,1):
            self.set_list() # store values
            print ('Day = %s'%(day+1))
            check1 = self.stock_5[-1].shape[0]-self.latest_day-self.pred_day*6
            check2 = self.stock_5[-1].shape[0]-self.latest_day-self.pred_day*2
            if (check1 > 0):
                for k in range(0,self.pred_day*5,self.pred_day):#9000-self.latest_day-600,self.pred_day):
                    i = k + self.stock_5[day].shape[0]-self.latest_day-self.pred_day*5
                    print ('--------------------Rolling Window Time = %s--------------------'%(k/pred_day))
                    # Train data
                    X_train = self.stock_5[day][i:i+self.latest_day]
                    data_train = self.rf_5[day][i:i+self.latest_day]

                    train_rise = data_train[:,-3]
                    train_fall = data_train[:,-2]
                    train_noise = data_train[:,-1]
                    y_train = train_rise

                    # Test data
                    X_test = self.stock_5[day][i + self.latest_day:i + self.latest_day + self.pred_day]
                    data_test = self.rf_5[day][i + self.latest_day:i + self.latest_day + self.pred_day]
                    test_rise = data_test[:,-3]
                    test_fall = data_test[:,-2]
                    test_noise = data_test[:,-1]
                    y_test = test_rise

                    #start = time.time()
                    self.Grid_fit(X_train, y_train, cv = 2, scoring = 'accuracy')
                    self.model_fit(X_train, y_train,X_test,y_test)
                    #end = time.time()
                    #print 'Total Time = %s'%(end - start)
            elif(check2 > 0):
                for k in range(0,self.pred_day*2,self.pred_day):#9000-self.latest_day-600,self.pred_day):
                    i = k + self.stock_5[day].shape[0]-self.latest_day-self.pred_day*2
                    print ('--------------------Rolling Window Time = %s--------------------'%(k/pred_day))
                    # Train data
                    X_train = self.stock_5[day][i:i+self.latest_day]
                    data_train = self.rf_5[day][i:i+self.latest_day]

                    train_rise = data_train[:,-3]
                    train_fall = data_train[:,-2]
                    train_noise = data_train[:,-1]
                    y_train = train_rise

                    # Test data
                    X_test = self.stock_5[day][i + self.latest_day:i + self.latest_day + self.pred_day]
                    data_test = self.rf_5[day][i + self.latest_day:i + self.latest_day + self.pred_day]
                    test_rise = data_test[:,-3]
                    test_fall = data_test[:,-2]
                    test_noise = data_test[:,-1]
                    y_test = test_rise

                    #start = time.time()
                    self.Grid_fit(X_train, y_train, cv = 2, scoring = 'accuracy')
                    self.model_fit(X_train, y_train,X_test,y_test)
                    #end = time.time()
                    #print 'Total Time = %s'%(end - start)
            else:
                
                i = 0
                print ('--------------------Rolling Window Time = %s--------------------'%(i/pred_day))
                # Train data
                X_train = self.stock_5[day][i:-1*self.pred_day]
                data_train = self.rf_5[day][i:-1*self.pred_day]

                train_rise = data_train[:,-3]
                train_fall = data_train[:,-2]
                train_noise = data_train[:,-1]
                y_train = train_rise

                # Test data
                X_test = self.stock_5[day][-1*self.pred_day:]
                data_test = self.rf_5[day][-1*self.pred_day:]
                test_rise = data_test[:,-3]
                test_fall = data_test[:,-2]
                test_noise = data_test[:,-1]
                y_test = test_rise

                #start = time.time()
                self.Grid_fit(X_train, y_train, cv = 2, scoring = 'accuracy')
                self.model_fit(X_train, y_train,X_test,y_test)
                #end = time.time()
                #print 'Total Time = %s'%(end - start)
                
            for key in self.keys:
                
                self.cv_acc_day[key].append(self.cv_acc[key])
                self.acc_day[key].append(self.acc[key])
                self.fscore_day[key].append(self.fscore[key])
                self.true_values_day[key].append(self.true_values[key])
                self.predict_values_day[key].append(self.predict_values[key])
            if (check2 > 0):
                self.summary_day.append(self.score_summary(sort_by = 'Accuracy_mean'))
            else:
                self.summary_day.append(self.score_summary_1cv(sort_by = 'Accuracy_mean'))
            
    def model_pre(self,X_train,y_train,X_test):
        for key in self.keys:
            print ("Running training & testing for %s." %(key))
            model = self.models[key]
            model.set_params(**self.grid[key].best_params_)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)
            #print 'Prediction latest 15 second = %s'%(predictions)
            self.predict_values[key].append(predictions.tolist())
            
        
    def pipline_pre(self):
        self.set_list_day() # store day values
        for day in range(0,self.day,1):
            self.set_list() # store values
            print ('Day = %s'%(day+1))
            
            # Train data
            X_train = self.stock_5[day]
            data_train = self.rf_5[day]
            
            train_rise = data_train[:,-3]
            train_fall = data_train[:,-2]
            train_noise = data_train[:,-1]
            y_train = train_rise
            
            #test
            X_test = self.stock_5[day][-1*window:,:]
            
            #start = time.time()
            self.Grid_fit(X_train, y_train, cv = 2, scoring = 'accuracy')
            self.model_pre(X_train, y_train , X_test)
            #end = time.time()
            #print 'Total Time = %s'%(end - start)
                
            for key in self.keys:
                
                self.cv_acc_day[key].append(self.cv_acc[key])
                self.acc_day[key].append(self.acc[key])
                self.fscore_day[key].append(self.fscore[key])
                self.true_values_day[key].append(self.true_values[key])
                self.predict_values_day[key].append(self.predict_values[key])
            
#             self.summary_day.append(self.score_summary(sort_by = 'Accuracy_mean'))
            
    
    
    def set_list(self):
        
        for key in self.keys:
            self.predict_values[key] = []
            self.cv_acc[key] = []
            self.acc[key] = []
            self.fscore[key] = []
            self.true_values[key] = []
            
    def set_list_day(self):
        
        for key in self.keys:
            self.predict_values_day[key] = []
            self.cv_acc_day[key] = []
            self.acc_day[key] = []
            self.fscore_day[key] = []
            self.true_values_day[key] = []
            
    def score_summary(self,sort_by):
        
        summary = pd.concat([pd.Series(list(self.acc.keys())),pd.Series(map(lambda x: sum(self.acc[x])/len(self.acc[x]), self.acc)),\
                             pd.Series(list(map(lambda x: statistics.stdev(self.acc[x]), self.acc))),\
                             pd.Series(list(map(lambda x: max(self.acc[x]), self.acc))),\
                             pd.Series(list(map(lambda x: min(self.acc[x]), self.acc))),\
                             pd.Series(list(map(lambda x: sum(self.fscore[x])/len(self.fscore[x]), self.fscore)))],axis=1)
        summary.columns = ['Estimator','Accuracy_mean','Accuracy_std','Accuracy_max','Accuracy_min','F_score']
        summary.index.rename('Ranking', inplace=True)
        return summary.sort_values(by = [sort_by], ascending=False)
    
    def score_summary_1cv(self,sort_by):
        
        summary = pd.concat([pd.Series(list(self.acc.keys())),pd.Series(map(lambda x: sum(self.acc[x])/len(self.acc[x]), self.acc)),\
                             
                             pd.Series(list(map(lambda x: max(self.acc[x]), self.acc))),\
                             pd.Series(list(map(lambda x: min(self.acc[x]), self.acc))),\
                             pd.Series(list(map(lambda x: sum(self.fscore[x])/len(self.fscore[x]), self.fscore)))],axis=1)
        summary.columns = ['Estimator','Accuracy_mean','Accuracy_max','Accuracy_min','F_score']
        summary.index.rename('Ranking', inplace=True)
        return summary.sort_values(by = [sort_by], ascending=False)
          
    def print_(self):

        print (self.predict_values)

In [12]:
def get_dataframe_drop_5(df_list,df_r,target,interval,window,ca=4,day = 1):
    df_list_rate = []
    df_rf = get_rise_fall(target,interval).shift(-1*day+1)
    mrf = pd.Series(get_mean_rf(df_list[0].shift(1) ,'close',20,5),index=df_list[0].index)
    #concat the df in the df_list
    for index,df in enumerate(df_list):
        if(df_r[index]):
            df_rate = get_change_rate(df,window).iloc[:,df.shape[1]:]
        else:
            df_rate = get_backday(df,window).iloc[:,df.shape[1]:]
            
        df_av  = get_av_ca(stock=df_rate,rf_df=df_rf,ca=ca,window = window).iloc[:,:-3]
        df_colname = list(map(lambda x:  df_name[index]+'_'+x,df_av.columns))
        df_dum = pd.get_dummies(df_av,prefix=df_colname,columns=df_av.columns)
        df_list_rate.append(df_dum)
        
    df_all = pd.concat(df_list_rate,axis =1)
    df_all['mean_rise_fall'] = mrf
    
    df_total = df_all.dropna(axis = 0,how = 'any')
    df_rf_ = df_rf.loc[df_total.index]
    #get target rise fall data
    Y = df_list[0].copy()
    Y['trf'] = np.sign((df_list[0]-df_list[0].shift(1))['close'].values)
    Y = Y.shift(-1*day+1)
    trf = Y.loc[df_total.index]['trf']

    return [df_total,trf,df_rf_]

In [13]:
ETF701 = extract_target_ETF(ETFtable,701) 

In [14]:
aa = get_change_rate(ETF701,30)

In [15]:
aa.shape

(158, 155)

In [16]:
def get_week_score(df_score):
    sum_ = 0
    propotion = [0.1,0.15,0.2,0.25,0.3]
    for day,col in enumerate(df_score.columns):
        sum_ += df_score[col].sum()*propotion[day]
    return sum_

# CV Prediction 

In [17]:
ETF_list = [50,51,52,53,54,55,56,57,58,59,6201,6203,6204,6208,690,692,701,713]

## all data without df 

In [13]:
interval = 0
window = 1
ETF_week_score = []

for code in ETF_list:
    ETF_price,ETF_rate,mrf = get_targetdata(ETFtable,code)
    df_list = [ETF_price,orderbook,bond,TXF,EXF,FXF,E_F,Nikkei,SP,Russell,DJ,VIX,pcr,OI]
    df_option = [True,True,True,True,True,True,False,True,True,True,True,True,False,True]
    df_name = ['Yunta50','orderbook','bond','TXF','EXF','FXF','E_F','Nikkei','SP','Russell','DJ','VIX','pcr','OI']
    print('Code:',code)
    #get day data
    stock_5 = []
    trf_5 = []
    for day in range(5):
        df_total ,trf = get_dataframe(df_list,df_option,ETF_rate,interval,window,ca=4,day = day+1)
        stock_5.append(df_total.as_matrix())
        trf_5.append(trf.values)
    
    #train and test  
    pred_day = stock_5[0].shape[0]//20
    latest_day = (stock_5[0].shape[0]-pred_day*6)-2
    day = 5
    pip = Model_Selection(models,model_grid_params,stock_5,latest_day,pred_day,day)
    pip.pipline()
    
    #record score
    week_score = []
    for day in range(5):
        score = pip.summary_day[day]['Accuracy_mean'].mean()*0.5
        week_score.append(score)
    
    ETF_week_score.append(week_score)
df_score = pd.DataFrame(ETF_week_score,columns=['day1','day2','day3','day4','day5'])
df_score.index = ETF_list

Code: 50
Day = 1
--------------------Rolling Window Time = 0.0--------------------
Running GridSearchCV for DecisionTreeClassifier
{'max_depth': 1, 'min_samples_leaf': 1}
CV Best Score = 0.5883002207505519
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.5938189845474614
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.6875
Running training & testing for RandomForestClassifier.
Accuracy = 0.65625
--------------------Rolling Window Time = 1.0--------------------
Running GridSearchCV for DecisionTreeClassifier
{'max_depth': 5, 'min_samples_leaf': 1}
CV Best Score = 0.5827814569536424
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.5949227373068433
Running training & testing for D

Accuracy = 0.546875
--------------------Rolling Window Time = 3.0--------------------
Running GridSearchCV for DecisionTreeClassifier
{'max_depth': 1, 'min_samples_leaf': 1}
CV Best Score = 0.5132450331125827
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 5}
CV Best Score = 0.5187637969094923
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.4375
Running training & testing for RandomForestClassifier.
Accuracy = 0.40625
--------------------Rolling Window Time = 4.0--------------------
Running GridSearchCV for DecisionTreeClassifier
{'max_depth': 5, 'min_samples_leaf': 5}
CV Best Score = 0.4977924944812362
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.49448123620309054
Running training & testing f

Code: 51
Day = 1
--------------------Rolling Window Time = 0.0--------------------
Running GridSearchCV for DecisionTreeClassifier
{'max_depth': 1, 'min_samples_leaf': 1}
CV Best Score = 0.5562913907284768
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.5584988962472406
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.515625
Running training & testing for RandomForestClassifier.
Accuracy = 0.53125
--------------------Rolling Window Time = 1.0--------------------
Running GridSearchCV for DecisionTreeClassifier
{'max_depth': 1, 'min_samples_leaf': 1}
CV Best Score = 0.5463576158940397
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.5728476821192053
Running training & testing for

{'max_depth': None, 'min_samples_leaf': 1}
CV Best Score = 0.5044150110375276
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.4966887417218543
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.5625
Running training & testing for RandomForestClassifier.
Accuracy = 0.5
--------------------Rolling Window Time = 4.0--------------------
Running GridSearchCV for DecisionTreeClassifier
{'max_depth': 5, 'min_samples_leaf': 5}
CV Best Score = 0.5176600441501104
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 5}
CV Best Score = 0.5176600441501104
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.578125
Running training & testing for RandomForestClassifier.
Accuracy = 0.5
-------------------

Day = 1
--------------------Rolling Window Time = 0.0--------------------
Running GridSearchCV for DecisionTreeClassifier
{'max_depth': 1, 'min_samples_leaf': 1}
CV Best Score = 0.6026490066225165
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.6158940397350994
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.75
Running training & testing for RandomForestClassifier.
Accuracy = 0.75
--------------------Rolling Window Time = 1.0--------------------
Running GridSearchCV for DecisionTreeClassifier
{'max_depth': 1, 'min_samples_leaf': 1}
CV Best Score = 0.5927152317880795
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.6169977924944813
Running training & testing for DecisionTreeCla

...

{'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.5317919075144508
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.5
Running training & testing for RandomForestClassifier.
Accuracy = 0.6
Code: 701
Day = 1
--------------------Rolling Window Time = 0.0--------------------
Running GridSearchCV for DecisionTreeClassifier
{'max_depth': None, 'min_samples_leaf': 1}
CV Best Score = 0.5634920634920635
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.5555555555555556
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.3333333333333333
Running training & testing for RandomForestClassifier.
Accuracy = 0.4444444444444444
--------------------Rolling Window Time = 1.0--------------------
Running GridSearchCV for DecisionTr

{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 5}
CV Best Score = 0.48412698412698413
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.6666666666666666
Running training & testing for RandomForestClassifier.
Accuracy = 0.5555555555555556
--------------------Rolling Window Time = 3.0--------------------
Running GridSearchCV for DecisionTreeClassifier
{'max_depth': None, 'min_samples_leaf': 2}
CV Best Score = 0.49206349206349204
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.48412698412698413
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.6666666666666666
Running training & testing for RandomForestClassifier.
Accuracy = 0.5555555555555556
--------------------Rolling Window Time = 4.0--------------------
Running GridSearchCV f

{'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.5555555555555556
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.7142857142857143
Running training & testing for RandomForestClassifier.
Accuracy = 0.7142857142857143
Code: 713
Day = 1
--------------------Rolling Window Time = 0.0--------------------
Running GridSearchCV for DecisionTreeClassifier
{'max_depth': None, 'min_samples_leaf': 10}
CV Best Score = 0.5871559633027523
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.5504587155963303
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.2857142857142857
Running training & testing for RandomForestClassifier.
Accuracy = 0.42857142857142855
--------------------Rolling Window Time = 1.0--------------------
Run

{'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 5}
CV Best Score = 0.5229357798165137
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.42857142857142855
Running training & testing for RandomForestClassifier.
Accuracy = 0.14285714285714285
--------------------Rolling Window Time = 3.0--------------------
Running GridSearchCV for DecisionTreeClassifier
{'max_depth': None, 'min_samples_leaf': 2}
CV Best Score = 0.45871559633027525
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.47706422018348627
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.5714285714285714
Running training & testing for RandomForestClassifier.
Accuracy = 0.5714285714285714
--------------------Rolling Window Time = 4.0--------------------
Running GridSearchCV

{'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 5}
CV Best Score = 0.4954128440366973
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.4
Running training & testing for RandomForestClassifier.
Accuracy = 0.2


In [47]:
df_score.index = ETF_list
df_score['day5'].sum()

4.688846506223018

In [40]:
get_week_score(df_score)

4.692734071833744

# Data with dropping the null 

In [14]:
interval = 0
window = 1
ETF_week_score = []

for code in ETF_list:
    ETF_price,ETF_rate,mrf = get_targetdata(ETFtable,code)
    df_list = [ETF_price,orderbook,oil_price,bond,TXF,EXF,FXF,E_F,Nikkei,SP,Russell,DJ,SOX,VIX,pcr,FOI,IOI,DOI]
    df_option = [True,True,True,True,True,True,True,False,True,True,True,True,True,True,False,True,True,True]
    df_name = ['Yunta50','orderbook','oil_price','bond','TXF','EXF','FXF','E_F','Nikkei','SP','Russell','DJ','SOX','VIX','pcr','FOI','IOI','DOI']
    print('Code:',code)
    #get day data
    stock_5 = []
    trf_5 = []
    for day in range(5):
        df_total ,trf = get_dataframe_drop(df_list,df_option,ETF_rate,interval,window,ca=4,day = day+1)
        stock_5.append(df_total.as_matrix())
        trf_5.append(trf.values)
    
    #train and test  
    pred_day = stock_5[0].shape[0]//20
    latest_day = (stock_5[0].shape[0]-pred_day*6)-2
    day = 5
    pip = Model_Selection(models,model_grid_params,stock_5,latest_day,pred_day,day)
    pip.pipline()
    
    #record score
    week_score = []
    for day in range(5):
        score = pip.summary_day[day]['Accuracy_mean'].mean()*0.5
        week_score.append(score)
    
    ETF_week_score.append(week_score)
df_score = pd.DataFrame(ETF_week_score,columns=['day1','day2','day3','day4','day5'])
df_score.index = ETF_list

Code: 50


NameError: name 'Yunta50' is not defined

In [ ]:
get_week_score(df_score)

## Data with df 

In [16]:
interval = 0
window = 1
ETF_week_score = []

for code in ETF_list:
    ETF_price,ETF_rate,mrf = get_targetdata(ETFtable,code)
    df_list = [ETF_price,orderbook,bond,TXF,EXF,FXF,E_F,Nikkei,SP,Russell,DJ,VIX,pcr,OI]
    df_option = [True,True,True,True,True,True,False,True,True,True,True,True,False,True]
    df_name = ['Yunta50','orderbook','bond','TXF','EXF','FXF','E_F','Nikkei','SP','Russell','DJ','VIX','pcr','OI']
    print('Code:',code)
    #get day data
    
    df_total,trf,rf = get_dataframe_drop_5(df_list,df_option,ETF_rate,interval,window)
    rf_5 = [rf.iloc[:-4,:].as_matrix()]
    trf_5 = [trf.iloc[:-4].values]
    for day in range(4):
        rf_5.append(rf.shift(-1*day-1).iloc[:-4,:].as_matrix())
        trf_5.append(trf.shift(-1*day-1).iloc[:-4].values)
    stock_5 = [df_total.iloc[:-4,:].as_matrix(),df_total.iloc[:-4,:].as_matrix(),df_total.iloc[:-4,:].as_matrix(),df_total.iloc[:-4,:].as_matrix(),df_total.iloc[:-4,:].as_matrix()]

    #train and test  
    pred_day = rf_5[0].shape[0]//20
    latest_day = (rf_5[0].shape[0]-pred_day*6)-2
    day = 5
    pip = Model_Selection2(models,model_grid_params,stock_5,rf_5,latest_day,pred_day,day)
    pip.pipline()
    
    #record score
    week_score = []
    for day in range(5):
        score = pip.summary_day[day]['Accuracy_mean'].mean()*0.5
        week_score.append(score)
    
    ETF_week_score.append(week_score)
df_score_all = pd.DataFrame(ETF_week_score,columns=['day1','day2','day3','day4','day5'])
df_score_all.index = ETF_list

Code: 50
Day = 1
--------------------Rolling Window Time = 0.0--------------------
Running GridSearchCV for DecisionTreeClassifier
{'max_depth': 1, 'min_samples_leaf': 1}
CV Best Score = 0.6218181818181818
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 5}
CV Best Score = 0.5927272727272728
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.6440677966101694
Running training & testing for RandomForestClassifier.
Accuracy = 0.6101694915254238
--------------------Rolling Window Time = 1.0--------------------
Running GridSearchCV for DecisionTreeClassifier
{'max_depth': 1, 'min_samples_leaf': 1}
CV Best Score = 0.5842424242424242
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 5}
CV Best Score = 0.6
Running training & testing

{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.5224242424242425
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.5423728813559322
Running training & testing for RandomForestClassifier.
Accuracy = 0.423728813559322
--------------------Rolling Window Time = 3.0--------------------
Running GridSearchCV for DecisionTreeClassifier
{'max_depth': 1, 'min_samples_leaf': 1}
CV Best Score = 0.5127272727272727
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 5}
CV Best Score = 0.526060606060606
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.4915254237288136
Running training & testing for RandomForestClassifier.
Accuracy = 0.5254237288135594
--------------------Rolling Window Time = 4.0--------------------
Running GridSearchCV for Decis

{'max_depth': None, 'min_samples_leaf': 2}
CV Best Score = 0.5321212121212121
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.5078787878787879
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.5254237288135594
Running training & testing for RandomForestClassifier.
Accuracy = 0.4576271186440678
Code: 51
Day = 1
--------------------Rolling Window Time = 0.0--------------------
Running GridSearchCV for DecisionTreeClassifier
{'max_depth': 1, 'min_samples_leaf': 1}
CV Best Score = 0.5527272727272727
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 5}
CV Best Score = 0.5527272727272727
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.5423728813559322
Running training & testing for Ran

{'max_depth': 10, 'min_samples_leaf': 2}
CV Best Score = 0.5115151515151515
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.5042424242424243
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.5254237288135594
Running training & testing for RandomForestClassifier.
Accuracy = 0.4067796610169492
--------------------Rolling Window Time = 3.0--------------------
Running GridSearchCV for DecisionTreeClassifier
{'max_depth': None, 'min_samples_leaf': 10}
CV Best Score = 0.49818181818181817
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.49333333333333335
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.4406779661016949
Running training & testing for RandomForestClas

{'max_depth': 5, 'min_samples_leaf': 1}
CV Best Score = 0.5248484848484849
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.5066666666666667
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.5084745762711864
Running training & testing for RandomForestClassifier.
Accuracy = 0.423728813559322
Code: 52
Day = 1
--------------------Rolling Window Time = 0.0--------------------
Running GridSearchCV for DecisionTreeClassifier
{'max_depth': 1, 'min_samples_leaf': 1}
CV Best Score = 0.6303030303030303
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.6133333333333333
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.7457627118644068
Running training & testing for RandomF

...

{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 5}
CV Best Score = 0.49032258064516127
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.5
Running training & testing for RandomForestClassifier.
Accuracy = 0.5
--------------------Rolling Window Time = 3.0--------------------
Running GridSearchCV for DecisionTreeClassifier
{'max_depth': 5, 'min_samples_leaf': 2}
CV Best Score = 0.535483870967742
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.5419354838709678
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.4
Running training & testing for RandomForestClassifier.
Accuracy = 0.4
--------------------Rolling Window Time = 4.0--------------------
Running GridSearchCV for DecisionTreeClassifier
{'max_depth': 5, 'min_samples_leaf': 1}

{'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.509090909090909
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.7142857142857143
Running training & testing for RandomForestClassifier.
Accuracy = 0.7142857142857143
Day = 3
--------------------Rolling Window Time = 0.0--------------------
Running GridSearchCV for DecisionTreeClassifier
{'max_depth': 5, 'min_samples_leaf': 1}
CV Best Score = 0.5363636363636364
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 5}
CV Best Score = 0.5545454545454546
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.5714285714285714
Running training & testing for RandomForestClassifier.
Accuracy = 0.42857142857142855
--------------------Rolling Window Time = 1.0--------------------
Running GridSearch

{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.6272727272727273
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.42857142857142855
Running training & testing for RandomForestClassifier.
Accuracy = 0.42857142857142855
--------------------Rolling Window Time = 3.0--------------------
Running GridSearchCV for DecisionTreeClassifier
{'max_depth': None, 'min_samples_leaf': 1}
CV Best Score = 0.4909090909090909
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.5454545454545454
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.42857142857142855
Running training & testing for RandomForestClassifier.
Accuracy = 0.42857142857142855
--------------------Rolling Window Time = 4.0--------------------
Running GridSearchC

{'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 5}
CV Best Score = 0.6363636363636364
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.3333333333333333
Running training & testing for RandomForestClassifier.
Accuracy = 0.8333333333333334
Day = 3
--------------------Rolling Window Time = 0.0--------------------
Running GridSearchCV for DecisionTreeClassifier
{'max_depth': 1, 'min_samples_leaf': 1}
CV Best Score = 0.5568181818181818
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 5}
CV Best Score = 0.5340909090909091
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.3333333333333333
Running training & testing for RandomForestClassifier.
Accuracy = 0.16666666666666666
--------------------Rolling Window Time = 1.0--------------------
Running GridSearch

{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 5}
CV Best Score = 0.5113636363636364
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.0
Running training & testing for RandomForestClassifier.
Accuracy = 0.5
--------------------Rolling Window Time = 3.0--------------------
Running GridSearchCV for DecisionTreeClassifier
{'max_depth': None, 'min_samples_leaf': 2}
CV Best Score = 0.5454545454545454
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 5}
CV Best Score = 0.5454545454545454
Running training & testing for DecisionTreeClassifier.
Accuracy = 0.5
Running training & testing for RandomForestClassifier.
Accuracy = 0.16666666666666666
--------------------Rolling Window Time = 4.0--------------------
Running GridSearchCV for DecisionTreeClassifier
{'max_depth': 5, 'min_

In [17]:
get_week_score(df_score_all)

4.690967177831584

In [18]:
df_score_all['day1'].mean()

0.29545126744985495

##### Date with df and without df are almost the same 

# By American data only 

In [26]:
interval = 0
window = 1
ETF_week_score = []

for code in ETF_list:
    ETF_price,ETF_rate,mrf = get_targetdata(ETFtable,code)
    df_list = [ETF_price,SP,Russell,DJ,VIX]
    df_option = [True,True,True,True,True]
    df_name = ['Yunta50','SP','Russell','DJ','VIX']
    print('Code:',code)
    #get day data
    
    df_total,trf,rf = get_dataframe_drop_5(df_list,df_option,ETF_rate,interval,window)
    rf_5 = [rf.iloc[:-4,:].as_matrix()]
    trf_5 = [trf.iloc[:-4].values]
    for day in range(4):
        rf_5.append(rf.shift(-1*day-1).iloc[:-4,:].as_matrix())
        trf_5.append(trf.shift(-1*day-1).iloc[:-4].values)
    
    stock_5 = [df_total.iloc[:-4,:].as_matrix(),df_total.iloc[:-4,:].as_matrix(),df_total.iloc[:-4,:].as_matrix(),df_total.iloc[:-4,:].as_matrix(),df_total.iloc[:-4,:].as_matrix()]

    #train and test  
    pred_day = 10
    latest_day = 50
    day = 5
    pip = Model_Selection2(models,model_grid_params,stock_5,rf_5,latest_day,pred_day,day)
    pip.pipline()
    
    #record score
    week_score = []
    for day in range(5):
        score = pip.summary_day[day]['Accuracy_mean'].mean()*0.5
        week_score.append(score)
    
    ETF_week_score.append(week_score)
df_score_am = pd.DataFrame(ETF_week_score,columns=['day1','day2','day3','day4','day5'])
df_score_am.index = ETF_list

Code: 50
Day = 1
--------------------Rolling Window Time = 113.5--------------------
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.5
Running training & testing for RandomForestClassifier.
Accuracy = 0.6
--------------------Rolling Window Time = 114.5--------------------
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.54
Running training & testing for RandomForestClassifier.
Accuracy = 0.6
--------------------Rolling Window Time = 115.5--------------------
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.54
Running training & testing for RandomForest

{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 50}
CV Best Score = 0.66
Running training & testing for RandomForestClassifier.
Accuracy = 0.3
--------------------Rolling Window Time = 114.5--------------------
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.48
Running training & testing for RandomForestClassifier.
Accuracy = 0.5
--------------------Rolling Window Time = 115.5--------------------
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.5
Running training & testing for RandomForestClassifier.
Accuracy = 0.5
--------------------Rolling Window Time = 116.5--------------------
Running GridSearchCV for RandomForestC

{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 50}
CV Best Score = 0.48
Running training & testing for RandomForestClassifier.
Accuracy = 0.6
--------------------Rolling Window Time = 115.5--------------------
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.42
Running training & testing for RandomForestClassifier.
Accuracy = 0.4
--------------------Rolling Window Time = 116.5--------------------
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 50}
CV Best Score = 0.58
Running training & testing for RandomForestClassifier.
Accuracy = 0.3
--------------------Rolling Window Time = 117.5--------------------
Running GridSearchCV for RandomForest

{'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.44
Running training & testing for RandomForestClassifier.
Accuracy = 0.5
--------------------Rolling Window Time = 116.5--------------------
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.6
Running training & testing for RandomForestClassifier.
Accuracy = 0.3
--------------------Rolling Window Time = 117.5--------------------
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.6
Running training & testing for RandomForestClassifier.
Accuracy = 0.5
--------------------Rolling Window Time = 118.5--------------------
Running GridSearchCV for RandomForest

{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.52
Running training & testing for RandomForestClassifier.
Accuracy = 0.5
--------------------Rolling Window Time = 117.5--------------------
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 50}
CV Best Score = 0.54
Running training & testing for RandomForestClassifier.
Accuracy = 0.3
--------------------Rolling Window Time = 118.5--------------------
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 50}
CV Best Score = 0.52
Running training & testing for RandomForestClassifier.
Accuracy = 0.5
Day = 3
--------------------Rolling Window Time = 113.5--------------------
Running GridSearchCV for Ran

...

{'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 50}
CV Best Score = 0.58
Running training & testing for RandomForestClassifier.
Accuracy = 0.6
--------------------Rolling Window Time = 18.6--------------------
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 50}
CV Best Score = 0.68
Running training & testing for RandomForestClassifier.
Accuracy = 0.6
Day = 2
--------------------Rolling Window Time = 13.6--------------------
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.6
Running training & testing for RandomForestClassifier.
Accuracy = 0.4
--------------------Rolling Window Time = 14.6--------------------
Running GridSearchCV for Random

{'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.54
Running training & testing for RandomForestClassifier.
Accuracy = 0.7
Code: 692
Day = 1
--------------------Rolling Window Time = 10.7--------------------
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 50}
CV Best Score = 0.6
Running training & testing for RandomForestClassifier.
Accuracy = 0.7
--------------------Rolling Window Time = 11.7--------------------
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 50}
CV Best Score = 0.62
Running training & testing for RandomForestClassifier.
Accuracy = 0.5
--------------------Rolling Window Time = 12.7--------------------
Running GridSearchCV 

{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 50}
CV Best Score = 0.6
Running training & testing for RandomForestClassifier.
Accuracy = 0.5
--------------------Rolling Window Time = 11.7--------------------
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 50}
CV Best Score = 0.38
Running training & testing for RandomForestClassifier.
Accuracy = 0.3
--------------------Rolling Window Time = 12.7--------------------
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 50}
CV Best Score = 0.56
Running training & testing for RandomForestClassifier.
Accuracy = 0.4
--------------------Rolling Window Time = 13.7--------------------
Running GridSearchCV for RandomForestClas

{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 50}
CV Best Score = 0.62
Running training & testing for RandomForestClassifier.
Accuracy = 0.2
--------------------Rolling Window Time = 6.4--------------------
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 50}
CV Best Score = 0.48
Running training & testing for RandomForestClassifier.
Accuracy = 0.8
--------------------Rolling Window Time = 7.4--------------------
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.52
Running training & testing for RandomForestClassifier.
Accuracy = 0.3
--------------------Rolling Window Time = 8.4--------------------
Running GridSearchCV for RandomForestClassi

{'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 50}
CV Best Score = 0.6
Running training & testing for RandomForestClassifier.
Accuracy = 0.5
--------------------Rolling Window Time = 4.6--------------------
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.58
Running training & testing for RandomForestClassifier.
Accuracy = 0.4
--------------------Rolling Window Time = 5.6--------------------
Running GridSearchCV for RandomForestClassifier
{'criterion': 'entropy', 'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 50}
CV Best Score = 0.46
Running training & testing for RandomForestClassifier.
Accuracy = 0.5
--------------------Rolling Window Time = 6.6--------------------
Running GridSearchCV for RandomForestClassi

In [31]:
get_week_score(df_score_am)

4.5754166666666665

In [37]:
df_score_am['day5'].mean()

0.24398148148148147

In [32]:
df_score_am

,day1,day2,day3,day4,day5
50,0.275000,0.208333,0.250000,0.275000,0.216667
51,0.291667,0.275000,0.258333,0.275000,0.250000
52,0.275000,0.233333,0.191667,0.233333,0.258333
53,0.300000,0.258333,0.250000,0.291667,0.266667
54,0.308333,0.266667,0.275000,0.300000,0.283333
55,0.258333,0.225000,0.266667,0.208333,0.241667
56,0.283333,0.225000,0.275000,0.241667,0.225000
57,0.341667,0.233333,0.258333,0.233333,0.175000
58,0.291667,0.216667,0.241667,0.291667,0.166667
59,0.291667,0.200000,0.208333,0.241667,0.233333


# First day with American data 

In [28]:
interval = 0
ETF_week_score = []
start = time.time()
for code in ETF_list:
    ETF_price,ETF_rate,mrf = get_targetdata(ETFtable,code)
    df_list = [ETF_price]+features
    df_option = [True,True,True,True,True,True,False,True,True,True,True,True,False,True,True]
    df_name = ['Yunta50','orderbook','bond','TXF','EXF','FXF','E_F','Nikkei','SP','Russell','DJ','VIX','pcr','NAS','OI']
    print('Code:',code)
    #get day data
    
    df_total,trf,rf = get_dataframe_drop_5(df_list,df_option,ETF_rate,interval,window = 1)
    df_total2,trf2,rf2 = get_dataframe_drop_5(df_list,df_option,ETF_rate,interval,window = 2)
    df_total3,trf2,rf2 = get_dataframe_drop_5(df_list,df_option,ETF_rate,interval,window = 3)
    df_total4,trf2,rf2 = get_dataframe_drop_5(df_list,df_option,ETF_rate,interval,window = 4)
    df_total5,trf2,rf2 = get_dataframe_drop_5(df_list,df_option,ETF_rate,interval,window = 5)
    
    #extra
#     pcr_1 = get_change_rate(features[1],1).iloc[:,features[-3].shape[1]:]
#     bond_1 = get_change_rate(features[1],1).iloc[:,features[1].shape[1]:]
#     orderbook_50 = get_change_rate(features[0],50).iloc[:,features[0].shape[1]:]
#     orderbook_100 = get_change_rate(features[0],100).iloc[:,features[0].shape[1]:]
    
    df_total30,trf30,rf30 = get_dataframe_drop_5(df_list,df_option,ETF_rate,interval,window = 30)
    
#     df_total = pd.concat([df_total,orderbook_50,orderbook_100],axis = 1).dropna(axis = 0,how = 'any')
    df_total2 = pd.concat([df_total2,df_total30],axis = 1).dropna(axis = 0,how = 'any')
    df_total3 = pd.concat([df_total2,df_total30],axis = 1).dropna(axis = 0,how = 'any')
    df_total4 = pd.concat([df_total2,df_total30],axis = 1).dropna(axis = 0,how = 'any')
    df_total5 = pd.concat([df_total2,df_total30],axis = 1).dropna(axis = 0,how = 'any')
    
    diff = int(df_total.shape[0] - df_total2.shape[0])
    stock_5 = [df_total.iloc[diff:].as_matrix(),df_total2.shift(1).as_matrix(),df_total3.shift(2).as_matrix(),df_total4.shift(3).as_matrix(),df_total5.shift(4).as_matrix()]
    rf_diff = rf.iloc[diff:]
    rf_5 = [rf_diff.as_matrix(),rf_diff.as_matrix(),rf_diff.as_matrix(),rf_diff.as_matrix(),rf_diff.as_matrix()]
    
    #train and test  
    pred_day = 1
    latest_day = [30,30,30,30,30]
    day = 5
    pip = Model_Selection3(models,model_grid_params,stock_5,rf_5,latest_day,pred_day,day)
    pip.pipline()
    
    #record score
    week_score = []
    for day in range(5):
        score = pip.summary_day[day]['Accuracy_mean'].mean()*0.5
        week_score.append(score)
    
    ETF_week_score.append(week_score)
df_score_am_all = pd.DataFrame(ETF_week_score,columns=['day1','day2','day3','day4','day5'])
df_score_am_all.index = ETF_list
end = time.time()
print('Total time is %s minutes'%((end-start)/60))

Code: 50


KeyboardInterrupt: 

In [ ]:
# summary = pd.concat([pd.Series(list(pip.acc.keys())),pd.Series(map(lambda x: sum(pip.acc[x])/len(pip.acc[x]), pip.acc)),\
#                              pd.Series(list(map(lambda x: statistics.stdev(pip.acc[x]), pip.acc))),\
#                              pd.Series(list(map(lambda x: max(pip.acc[x]), pip.acc))),\
#                              pd.Series(list(map(lambda x: min(pip.acc[x]), pip.acc))),\
#                              pd.Series(list(map(lambda x: sum(pip.fscore[x])/len(pip.fscore[x]), pip.fscore)))],axis=1)

In [108]:
pip.acc_day

{'RandomForestClassifier': [[0.6333333333333333]]}

In [ ]:
#240 50  9 month
print(get_week_score(df_score_am_all))
print(df_score_am_all['day1'].mean())
print(df_score_am_all['day2'].mean())
print(df_score_am_all['day3'].mean())
print(df_score_am_all['day4'].mean())
print(df_score_am_all['day5'].mean())

In [123]:
#240 80  9 month
print(get_week_score(df_score_am_all))
print(df_score_am_all['day1'].mean())
print(df_score_am_all['day2'].mean())
print(df_score_am_all['day3'].mean())
print(df_score_am_all['day4'].mean())
print(df_score_am_all['day5'].mean())

4.757187500000001
0.29965277777777777
0.2567361111111111
0.2577083333333334
0.25875
0.2652777777777778


In [121]:
#150 100  9 month
print(get_week_score(df_score_am_all))
print(df_score_am_all['day1'].mean())
print(df_score_am_all['day2'].mean())
print(df_score_am_all['day3'].mean())
print(df_score_am_all['day4'].mean())
print(df_score_am_all['day5'].mean())

4.679399999999999
0.28855555555555557
0.24972222222222223
0.2607777777777778
0.2590555555555556
0.2557777777777777


In [119]:
#150 20  9 month
print(get_week_score(df_score_am_all))
print(df_score_am_all['day1'].mean())
print(df_score_am_all['day2'].mean())
print(df_score_am_all['day3'].mean())
print(df_score_am_all['day4'].mean())
print(df_score_am_all['day5'].mean())

4.605875
0.2940277777777778
0.2604166666666667
0.26430555555555557
0.23138888888888884
0.25569444444444445


In [117]:
#150 50  9 month
print(get_week_score(df_score_am_all))
print(df_score_am_all['day1'].mean())
print(df_score_am_all['day2'].mean())
print(df_score_am_all['day3'].mean())
print(df_score_am_all['day4'].mean())
print(df_score_am_all['day5'].mean())

4.7196
0.30011111111111105
0.25244444444444447
0.2582222222222222
0.2564444444444445
0.26188888888888884


In [114]:
#90 30  6 month
print(get_week_score(df_score_am_all))
print(df_score_am_all['day1'].mean())
print(df_score_am_all['day2'].mean())
print(df_score_am_all['day3'].mean())
print(df_score_am_all['day4'].mean())
print(df_score_am_all['day5'].mean())

4.703666666666667
0.28824074074074074
0.25092592592592594
0.2632407407407407
0.2625925925925926
0.2551851851851852


In [70]:
#60 20  3 month
print(get_week_score(df_score_am_all))
print(df_score_am_all['day1'].mean())
print(df_score_am_all['day2'].mean())
print(df_score_am_all['day3'].mean())
print(df_score_am_all['day4'].mean())
print(df_score_am_all['day5'].mean())

4.577875
0.2751388888888889
0.2515277777777778
0.2618055555555555
0.25222222222222224
0.24555555555555555


In [61]:
#20 10  1 month
print(get_week_score(df_score_am_all))
print(df_score_am_all['day1'].mean())
print(df_score_am_all['day2'].mean())
print(df_score_am_all['day3'].mean())
print(df_score_am_all['day4'].mean())
print(df_score_am_all['day5'].mean())


4.380833333333333
0.2689814814814815
0.23888888888888887
0.23935185185185187
0.2444444444444444
0.23888888888888893


In [145]:
df_score_am_all['day1'].mean()

0.3041152886138761

In [70]:
get_week_score(df_score_am_all)

4.70687903551251

In [138]:
get_week_score(df_score_am_all)

4.709381473634652

In [146]:
df_score_am_all['day3'].mean()

0.2572543008100918

In [106]:
df_score_am_all = pd.DataFrame(ETF_week_score,columns=['day1','day2'])
df_score_am_all['day2'].mean()

0.2531909475156602

In [53]:
df_score_am_all

,day1,day2,day3,day4,day5
50,0.258333,0.225000,0.266667,0.250000,0.241667
51,0.300000,0.208333,0.241667,0.258333,0.216667
52,0.275000,0.241667,0.208333,0.258333,0.191667
53,0.266667,0.225000,0.233333,0.258333,0.250000
54,0.266667,0.283333,0.333333,0.291667,0.266667
55,0.283333,0.266667,0.275000,0.241667,0.241667
56,0.233333,0.208333,0.208333,0.233333,0.200000
57,0.300000,0.225000,0.225000,0.225000,0.233333
58,0.200000,0.250000,0.216667,0.250000,0.208333
59,0.258333,0.258333,0.225000,0.200000,0.183333


In [57]:
start = time.time()

In [58]:
end = time.time()

In [59]:
print('Total time is %s minutes'%((end-start)/60))

Total time is 0.03605546553929647 minutes


In [21]:
features[-3].head()

,high,close
date,,
2007-01-02,19.63,19.63
2007-01-03,14.96,14.96
2007-01-04,11.75,11.75
2007-01-05,8.12,8.12
2007-01-08,4.39,4.39


In [24]:
get_change_rate(features[0],50).iloc[:,features[0].shape[1]:]

,ask_vol_1,ask_count_1,bid_vol_1,bid_count_1,ask_vol_2,ask_count_2,bid_vol_2,bid_count_2,ask_vol_3,ask_count_3,...,bid_vol_48,bid_count_48,ask_vol_49,ask_count_49,bid_vol_49,bid_count_49,ask_vol_50,ask_count_50,bid_vol_50,bid_count_50
date,,,,,,,,,,,,,,,,,,,,,
2005-09-23,0.130342,0.130906,-0.234393,-0.152502,0.021251,0.068133,0.274252,0.228260,-0.013812,-0.021172,...,-0.050095,-0.016474,-0.071322,-0.121902,0.071509,0.083254,-0.019708,0.010440,0.117342,0.061442
2005-09-26,-0.035468,-0.198288,-0.118434,-0.186640,0.130342,0.130906,-0.234393,-0.152502,0.021251,0.068133,...,0.076625,0.009211,0.126427,0.112523,-0.050095,-0.016474,-0.071322,-0.121902,0.071509,0.083254
2005-09-27,0.000711,0.036227,-0.056768,0.119861,-0.035468,-0.198288,-0.118434,-0.186640,0.130342,0.130906,...,-0.383899,-0.236576,-0.041305,0.022664,0.076625,0.009211,0.126427,0.112523,-0.050095,-0.016474
2005-09-28,-0.083711,-0.117226,0.149921,0.042721,0.000711,0.036227,-0.056768,0.119861,-0.035468,-0.198288,...,0.190113,0.129443,-0.237007,-0.189027,-0.383899,-0.236576,-0.041305,0.022664,0.076625,0.009211
2005-09-29,0.027078,0.028400,-0.069419,-0.032450,-0.083711,-0.117226,0.149921,0.042721,0.000711,0.036227,...,0.047256,0.174627,0.214234,0.147400,0.190113,0.129443,-0.237007,-0.189027,-0.383899,-0.236576
2005-09-30,0.030809,0.064860,0.020544,-0.012289,0.027078,0.028400,-0.069419,-0.032450,-0.083711,-0.117226,...,0.031982,-0.139118,0.037688,0.092700,0.047256,0.174627,0.214234,0.147400,0.190113,0.129443
2005-10-03,0.115987,0.160508,0.191529,0.164570,0.030809,0.064860,0.020544,-0.012289,0.027078,0.028400,...,-0.165826,-0.053281,0.000221,-0.055054,0.031982,-0.139118,0.037688,0.092700,0.047256,0.174627
2005-10-04,-0.113796,-0.056867,0.092772,0.256314,0.115987,0.160508,0.191529,0.164570,0.030809,0.064860,...,0.170687,0.206650,0.158060,0.422141,-0.165826,-0.053281,0.000221,-0.055054,0.031982,-0.139118
2005-10-05,0.069483,-0.012875,-0.013846,-0.139304,-0.113796,-0.056867,0.092772,0.256314,0.115987,0.160508,...,-0.208042,-0.402137,-0.168118,-0.675468,0.170687,0.206650,0.158060,0.422141,-0.165826,-0.053281


In [55]:
interval = 0
ETF_week_score = []
start = time.time()
for code in ETF_list:
    ETF_price,ETF_rate,mrf = get_targetdata(ETFtable,code)
    df_list = [ETF_price]+features
    df_option = [True,True,True,True,True,True,False,True,True,True,True,True,False,True,True]
    df_name = ['Yunta50','orderbook','bond','TXF','EXF','FXF','E_F','Nikkei','SP','Russell','DJ','VIX','pcr','NAS','OI']
    print('Code:',code)
    #get day data
    
    df_total,trf,rf = get_dataframe_drop_5(df_list,df_option,ETF_rate,interval,window = 1)
    df_total2,trf2,rf2 = get_dataframe_drop_5(df_list,df_option,ETF_rate,interval,window = 2)
    df_total3,trf3,rf3 = get_dataframe_drop_5(df_list,df_option,ETF_rate,interval,window = 3)
    
    #extra
    pcr_1 = get_change_rate(features[1],1).iloc[:,features[-3].shape[1]:]
    bond_1 = get_change_rate(features[1],1).iloc[:,features[1].shape[1]:]
    orderbook_50 = get_change_rate(features[0],50).iloc[:,features[0].shape[1]:]
    orderbook_100 = get_change_rate(features[0],100).iloc[:,features[0].shape[1]:]
    
    df_total30,trf30,rf30 = get_dataframe_drop_5(df_list,df_option,ETF_rate,interval,window = 30)
    
#     df_total = pd.concat([df_total,orderbook_50,orderbook_100],axis = 1).dropna(axis = 0,how = 'any')
    df_total2 = pd.concat([df_total2,df_total30,pcr_1,bond_1,orderbook_50,orderbook_100],axis = 1).dropna(axis = 0,how = 'any')
#     df_total3 = pd.concat([df_total2,df_total30,pcr_1,bond_1,orderbook_50,orderbook_100],axis = 1).dropna(axis = 0,how = 'any')
    df_total3 = pd.concat([df_total3,df_total30],axis = 1).dropna(axis = 0,how = 'any')
    df_total4 = pd.concat([df_total2,df_total30,pcr_1,bond_1,orderbook_50,orderbook_100],axis = 1).dropna(axis = 0,how = 'any')
    df_total5 = pd.concat([df_total2,df_total30,pcr_1,bond_1,orderbook_50,orderbook_100],axis = 1).dropna(axis = 0,how = 'any')
    
    diff = int(df_total.shape[0] - df_total2.shape[0])
    stock_5 = [df_total.iloc[diff:].as_matrix(),df_total2.shift(1).as_matrix(),df_total3.shift(2).as_matrix(),df_total4.shift(3).as_matrix(),df_total5.shift(4).as_matrix()]
    rf_diff = rf.iloc[diff:]
    rf_5 = [rf_diff.as_matrix(),rf_diff.as_matrix(),rf_diff.as_matrix(),rf_diff.as_matrix(),rf_diff.as_matrix()]
    
    #train and test  
    pred_day = 1
    latest_day = [15,15,15,15,15]
    day = 5
    pip = Model_Selection3(models,model_grid_params,stock_5,rf_5,latest_day,pred_day,day)
    pip.pipline()
    
    #record score
    week_score = []
    for day in range(5):
        score = pip.summary_day[day]['Accuracy_mean'].mean()*0.5
        week_score.append(score)
    
    ETF_week_score.append(week_score)
df_score_am_all = pd.DataFrame(ETF_week_score,columns=['day1','day2','day3','day4','day5'])
df_score_am_all.index = ETF_list
end = time.time()
print('Total time is %s minutes'%((end-start)/60))

Code: 50
Day = 1
--------------------Rolling Window Time = 0.0--------------------
164
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.7333333333333333
Running training & testing for XGBClassifier.
Accuracy = 0.0
--------------------Rolling Window Time = 1.0--------------------
179
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.4
Running training & testing for XGBClassifier.
Accuracy = 1.0
--------------------Rolling Window Time = 2.0--------------------
194
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.4
Running training & testing for XGBClassifier.
Accuracy = 1.0
--------------------Rolling Window Time = 3.0--------------------
209
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.4666666666666667
Running training & testing for XGBClassifier.
Accuracy = 1.0
--------------------Rolling Window Time = 4.0

{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.6
Running training & testing for XGBClassifier.
Accuracy = 1.0
--------------------Rolling Window Time = 36.0--------------------
704
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.6
Running training & testing for XGBClassifier.
Accuracy = 1.0
--------------------Rolling Window Time = 37.0--------------------
719
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.7333333333333333
Running training & testing for XGBClassifier.
Accuracy = 0.0
--------------------Rolling Window Time = 38.0--------------------
734
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.6666666666666666
Running training & testing for XGBClassifier.
Accuracy = 1.0
--------------------Rolling Window Time = 39.0--------------------
749
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.666

{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.4666666666666667
Running training & testing for XGBClassifier.
Accuracy = 1.0
--------------------Rolling Window Time = 4.0--------------------
224
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.6
Running training & testing for XGBClassifier.
Accuracy = 0.0
--------------------Rolling Window Time = 5.0--------------------
239
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.7333333333333333
Running training & testing for XGBClassifier.
Accuracy = 1.0
--------------------Rolling Window Time = 6.0--------------------
254
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.6
Running training & testing for XGBClassifier.
Accuracy = 0.0
--------------------Rolling Window Time = 7.0--------------------
269
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.6666666

{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.6666666666666666
Running training & testing for XGBClassifier.
Accuracy = 1.0
--------------------Rolling Window Time = 39.0--------------------
749
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.6666666666666666
Running training & testing for XGBClassifier.
Accuracy = 1.0
--------------------Rolling Window Time = 40.0--------------------
764
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.6
Running training & testing for XGBClassifier.
Accuracy = 0.0
--------------------Rolling Window Time = 41.0--------------------
779
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.6666666666666666
Running training & testing for XGBClassifier.
Accuracy = 1.0
--------------------Rolling Window Time = 42.0--------------------
794
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Bes

{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.6
Running training & testing for XGBClassifier.
Accuracy = 0.0
--------------------Rolling Window Time = 7.0--------------------
269
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.6666666666666666
Running training & testing for XGBClassifier.
Accuracy = 0.0
--------------------Rolling Window Time = 8.0--------------------
284
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.5333333333333333
Running training & testing for XGBClassifier.
Accuracy = 1.0
--------------------Rolling Window Time = 9.0--------------------
299
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.6666666666666666
Running training & testing for XGBClassifier.
Accuracy = 1.0
--------------------Rolling Window Time = 10.0--------------------
314
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best S

...

{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.6
Running training & testing for XGBClassifier.
Accuracy = 0.0
--------------------Rolling Window Time = 56.0--------------------
1004
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.5333333333333333
Running training & testing for XGBClassifier.
Accuracy = 1.0
--------------------Rolling Window Time = 57.0--------------------
1019
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.6
Running training & testing for XGBClassifier.
Accuracy = 0.0
--------------------Rolling Window Time = 58.0--------------------
1034
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.6666666666666666
Running training & testing for XGBClassifier.
Accuracy = 0.0
--------------------Rolling Window Time = 59.0--------------------
1049
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0

{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.6
Running training & testing for XGBClassifier.
Accuracy = 1.0
--------------------Rolling Window Time = 4.0--------------------
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.6
Running training & testing for XGBClassifier.
Accuracy = 1.0
--------------------Rolling Window Time = 5.0--------------------
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.6
Running training & testing for XGBClassifier.
Accuracy = 0.0
--------------------Rolling Window Time = 6.0--------------------
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.5333333333333333
Running training & testing for XGBClassifier.
Accuracy = 0.0
Day = 5
--------------------Rolling Window Time = 0.0--------------------
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.4666666666666667
Running train

{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.4666666666666667
Running training & testing for XGBClassifier.
Accuracy = 1.0
--------------------Rolling Window Time = 4.0--------------------
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.6
Running training & testing for XGBClassifier.
Accuracy = 1.0
--------------------Rolling Window Time = 5.0--------------------
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.5333333333333333
Running training & testing for XGBClassifier.
Accuracy = 0.0
--------------------Rolling Window Time = 6.0--------------------
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.6
Running training & testing for XGBClassifier.
Accuracy = 0.0
Day = 5
--------------------Rolling Window Time = 0.0--------------------
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.466666666666666

{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.6
Running training & testing for XGBClassifier.
Accuracy = 0.0
--------------------Rolling Window Time = 5.0--------------------
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.6
Running training & testing for XGBClassifier.
Accuracy = 1.0
--------------------Rolling Window Time = 6.0--------------------
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.6
Running training & testing for XGBClassifier.
Accuracy = 0.0
Day = 5
--------------------Rolling Window Time = 0.0--------------------
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.4
Running training & testing for XGBClassifier.
Accuracy = 1.0
--------------------Rolling Window Time = 1.0--------------------
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.6666666666666666
Running training & testing f

{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.4666666666666667
Running training & testing for XGBClassifier.
Accuracy = 1.0
--------------------Rolling Window Time = 5.0--------------------
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.6
Running training & testing for XGBClassifier.
Accuracy = 0.0
--------------------Rolling Window Time = 6.0--------------------
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.6666666666666666
Running training & testing for XGBClassifier.
Accuracy = 0.0
Day = 5
--------------------Rolling Window Time = 0.0--------------------
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.6666666666666666
Running training & testing for XGBClassifier.
Accuracy = 1.0
--------------------Rolling Window Time = 1.0--------------------
Running GridSearchCV for XGBClassifier
{'max_depth': 2, 'n_estimators': 10}
CV Best Score = 0.

In [43]:
stock_5[-1].shape

(1180, 478)

In [41]:
rf_5[3].shape

(1184, 3)

In [72]:
df_score_am_all

,day1,day2,day3,day4,day5
50,0.286765,0.2550,0.316176,0.2350,0.2775
51,0.301471,0.2450,0.250000,0.2575,0.2425
52,0.294118,0.2650,0.250000,0.3025,0.2600
53,0.286765,0.2750,0.345588,0.2700,0.3000
54,0.220588,0.2550,0.242647,0.2575,0.2450
55,0.308824,0.2450,0.242647,0.2550,0.2675
56,0.272059,0.2525,0.294118,0.2600,0.2700
57,0.286765,0.2500,0.286765,0.2675,0.2675
58,0.316176,0.2925,0.250000,0.2850,0.3075
59,0.250000,0.2875,0.227941,0.2675,0.2450


In [54]:
#20 20 20 20 20 xgboost thu good
print(get_week_score(df_score_am_all))
print(df_score_am_all['day1'].mean())
print(df_score_am_all['day2'].mean())
print(df_score_am_all['day3'].mean())
print(df_score_am_all['day4'].mean())
print(df_score_am_all['day5'].mean())

4.602
0.26333333333333336
0.2283333333333333
0.28555555555555556
0.24388888888888893
0.2566666666666667


In [52]:
#10 10 10 10 10 xgboost thu good
print(get_week_score(df_score_am_all))
print(df_score_am_all['day1'].mean())
print(df_score_am_all['day2'].mean())
print(df_score_am_all['day3'].mean())
print(df_score_am_all['day4'].mean())
print(df_score_am_all['day5'].mean())

4.9485
0.2808333333333333
0.2808333333333333
0.26416666666666666
0.2925
0.2625


In [50]:
#30 30 30 30 30 xgboost
print(get_week_score(df_score_am_all))
print(df_score_am_all['day1'].mean())
print(df_score_am_all['day2'].mean())
print(df_score_am_all['day3'].mean())
print(df_score_am_all['day4'].mean())
print(df_score_am_all['day5'].mean())

4.661764705882353
0.2900326797385621
0.22998366013071891
0.30310457516339867
0.2463235294117647
0.24428104575163395


In [48]:
#30 10 30 10 10 xgboost
print(get_week_score(df_score_am_all))
print(df_score_am_all['day1'].mean())
print(df_score_am_all['day2'].mean())
print(df_score_am_all['day3'].mean())
print(df_score_am_all['day4'].mean())
print(df_score_am_all['day5'].mean())

5.105235294117648
0.2900326797385621
0.2808333333333333
0.30310457516339867
0.2925
0.2625


In [33]:
#30 10 30 10 10 latest version adaboost
print(get_week_score(df_score_am_all))
print(df_score_am_all['day1'].mean())
print(df_score_am_all['day2'].mean())
print(df_score_am_all['day3'].mean())
print(df_score_am_all['day4'].mean())
print(df_score_am_all['day5'].mean())

4.929014705882353
0.28880718954248363
0.26333333333333336
0.2928921568627451
0.2708333333333333
0.26388888888888895


In [38]:
#30 10 30 10 10 latest version extra tree
print(get_week_score(df_score_am_all))
print(df_score_am_all['day1'].mean())
print(df_score_am_all['day2'].mean())
print(df_score_am_all['day3'].mean())
print(df_score_am_all['day4'].mean())
print(df_score_am_all['day5'].mean())

4.986073529411765
0.28880718954248363
0.2661111111111111
0.2990196078431373
0.27361111111111114
0.26666666666666666


In [47]:
#30 10 30 10 10 random forest
print(get_week_score(df_score_am_all))
print(df_score_am_all['day1'].mean())
print(df_score_am_all['day2'].mean())
print(df_score_am_all['day3'].mean())
print(df_score_am_all['day4'].mean())
print(df_score_am_all['day5'].mean())

5.267029411764706
0.3263888888888889
0.2986111111111111
0.3051470588235294
0.2752777777777778
0.28444444444444444


In [66]:
#30 1  rolling interval is self.latest_day
print(get_week_score(df_score_am_all))
print(df_score_am_all['day1'].mean())
print(df_score_am_all['day2'].mean())
print(df_score_am_all['day3'].mean())
print(df_score_am_all['day4'].mean())
print(df_score_am_all['day5'].mean())

5.048529411764706
0.3370098039215686
0.2708333333333333
0.3051470588235294
0.2790032679738562
0.2512254901960784


In [31]:
#30 1  WITH EXTRA FEATURE

print(get_week_score(df_score_am_all))
print(df_score_am_all['day1'].mean())
print(df_score_am_all['day2'].mean())
print(df_score_am_all['day3'].mean())
print(df_score_am_all['day4'].mean())
print(df_score_am_all['day5'].mean())

4.504411764705883
0.24673202614379086
0.24428104575163395
0.2487745098039216
0.23120915032679734
0.2712418300653595


In [51]:
#20 1  rolling interval is self.latest_day
# wed thu fri is good
print(get_week_score(df_score_am_all))
print(df_score_am_all['day1'].mean())
print(df_score_am_all['day2'].mean())
print(df_score_am_all['day3'].mean())
print(df_score_am_all['day4'].mean())
print(df_score_am_all['day5'].mean())

4.857
0.3011111111111111
0.2416666666666667
0.2816666666666667
0.28055555555555556
0.25666666666666665


In [28]:
#20 1 with extra features
# fri
print(get_week_score(df_score_am_all))
print(df_score_am_all['day1'].mean())
print(df_score_am_all['day2'].mean())
print(df_score_am_all['day3'].mean())
print(df_score_am_all['day4'].mean())
print(df_score_am_all['day5'].mean())

4.777
0.27722222222222226
0.23000000000000004
0.28
0.2466666666666667
0.2849999999999999


In [35]:
#15 1 with extra features
# 
print(get_week_score(df_score_am_all))
print(df_score_am_all['day1'].mean())
print(df_score_am_all['day2'].mean())
print(df_score_am_all['day3'].mean())
print(df_score_am_all['day4'].mean())
print(df_score_am_all['day5'].mean())

4.590191897654584
0.26901208244491825
0.25562662876095715
0.24312959014451552
0.2550343520492774
0.2579365079365079


In [62]:
#15 1  rolling interval is self.latest_day
# mon tue is good
print(get_week_score(df_score_am_all))
print(df_score_am_all['day1'].mean())
print(df_score_am_all['day2'].mean())
print(df_score_am_all['day3'].mean())
print(df_score_am_all['day4'].mean())
print(df_score_am_all['day5'].mean())

4.696215351812367
0.2703150912106136
0.24295190713101164
0.27789623312011374
0.250118455342336
0.26439232409381663


In [37]:
#10 1 with extra features
# tue fri
print(get_week_score(df_score_am_all))
print(df_score_am_all['day1'].mean())
print(df_score_am_all['day2'].mean())
print(df_score_am_all['day3'].mean())
print(df_score_am_all['day4'].mean())
print(df_score_am_all['day5'].mean())

5.0995
0.26972222222222225
0.2986111111111111
0.2869444444444445
0.2752777777777778
0.28444444444444444


In [64]:
#10 1  rolling interval is self.latest_day
print(get_week_score(df_score_am_all))
print(df_score_am_all['day1'].mean())
print(df_score_am_all['day2'].mean())
print(df_score_am_all['day3'].mean())
print(df_score_am_all['day4'].mean())
print(df_score_am_all['day5'].mean())

4.816
0.28444444444444444
0.27694444444444444
0.2725
0.25527777777777777
0.26416666666666666


In [41]:
#5 1 with extra features
# tue fri
print(get_week_score(df_score_am_all))
print(df_score_am_all['day1'].mean())
print(df_score_am_all['day2'].mean())
print(df_score_am_all['day3'].mean())
print(df_score_am_all['day4'].mean())
print(df_score_am_all['day5'].mean())

4.465000000000001
0.24805555555555558
0.24805555555555558
0.24805555555555558
0.24805555555555558
0.24805555555555558


In [71]:
#5 1  rolling interval is self.latest_day
print(get_week_score(df_score_am_all))
print(df_score_am_all['day1'].mean())
print(df_score_am_all['day2'].mean())
print(df_score_am_all['day3'].mean())
print(df_score_am_all['day4'].mean())
print(df_score_am_all['day5'].mean())

4.465000000000001
0.24805555555555558
0.24805555555555558
0.24805555555555558
0.24805555555555558
0.24805555555555558


In [45]:
np.array(pip.cv_acc_day['RandomForestClassifier'][4]).mean()

0.5899999999999999

# CV in split model 

# Prediction

In [ ]:
get_true_dataframe([Yunta50]+features,df_option,df_name,Yunta50_rate,0,1).iloc[-1]

In [205]:
interval = 0
window = 1
ETF_pre_1= []
ETF_pre_2= []


for code in ETF_list:
    ETF_price,ETF_rate = get_targetdata(ETFtable,code)
    df_list = [ETF_price] + features
    df_option = [True,True,True,True,True,True,False,True,True,True,True,True,False,True]
    df_name = ['Yunta50','orderbook','bond','TXF','EXF','FXF','E_F','Nikkei','SP','Russell','DJ','VIX','pcr','OI']
    print('Code:',code)
    #get day data
    
    df_total,trf,rf = get_dataframe_drop_5(df_list,df_option,ETF_rate,interval,window)
    df_total2,trf2,rf2 = get_dataframe_drop_5(df_list,df_option,ETF_rate,interval,window = 2)
    df_total3,trf3,rf3 = get_dataframe_drop_5(df_list,df_option,ETF_rate,interval,window = 3)
    df_total4,trf4,rf4 = get_dataframe_drop_5(df_list,df_option,ETF_rate,interval,window = 4)
    df_total5,trf5,rf5 = get_dataframe_drop_5(df_list,df_option,ETF_rate,interval,window = 5)
    
    df_total30,trf30,rf30 = get_dataframe_drop_5(df_list,df_option,ETF_rate,interval,window = 30)
    
    df_total2 = pd.concat([df_total2,df_total30],axis = 1).dropna(axis = 0,how = 'any')
    df_total3 = pd.concat([df_total3,df_total30],axis = 1).dropna(axis = 0,how = 'any')
    df_total4 = pd.concat([df_total4,df_total30],axis = 1).dropna(axis = 0,how = 'any')
    df_total5 = pd.concat([df_total5,df_total30],axis = 1).dropna(axis = 0,how = 'any')
    
    stock_5 = [df_total.as_matrix(),df_total2.shift(1).as_matrix(),df_total3.shift(2).as_matrix(),df_total4.shift(3).as_matrix(),df_total5.shift(4).as_matrix()]
    rf_5 = [rf.as_matrix(),rf.as_matrix(),rf.as_matrix(),rf.as_matrix(),rf.as_matrix()]
    
#     true_5 = get_true_dataframe([ETF_price]+features,df_option,df_name,ETF_rate,interval,window).as_matrix()
    #train and test  
    pred_day = 1
    latest_day = [15,15,20,20,20]
    day = 5
    pip = Model_Selection3(models,model_grid_params,stock_5,rf_30,latest_day,pred_day,day)
    pip.pipline_pre()
    
    #prediction
    key_pre = []
    for key in pip.keys:
        pre_ = np.array(pip.predict_values_day[key]).reshape((5))
        pre_t = np.array([-1,-1,-1,-1,-1])
        pre = pre_*2 + pre_t
        key_pre.append(pre)
    
    ETF_pre_1.append(key_pre[0])
#     ETF_pre_2.append(key_pre[1])
    
df_pre_1 = pd.DataFrame(ETF_pre_1,columns=['day1','day2','day3','day4','day5'])
df_pre_1.index = ETF_list
# df_pre_2 = pd.DataFrame(ETF_pre_2,columns=['day1','day2','day3','day4','day5'])
# df_pre_2.index = ETF_list


ValueError: too many values to unpack (expected 2)

In [46]:
np.array(pip.predict_values_day['DecisionTreeClassifier']).reshape((5))

array([0., 0., 0., 1., 0.])

In [131]:
df_pre_1

,day1,day2,day3,day4,day5
50,-1.0,-1.0,-1.0,1.0,1.0
51,-1.0,-1.0,-1.0,1.0,1.0
52,-1.0,-1.0,-1.0,-1.0,1.0
53,-1.0,1.0,-1.0,1.0,1.0
54,-1.0,1.0,-1.0,-1.0,1.0
55,-1.0,1.0,1.0,1.0,-1.0
56,1.0,1.0,-1.0,1.0,-1.0
57,-1.0,-1.0,-1.0,-1.0,1.0
58,-1.0,-1.0,-1.0,-1.0,-1.0
59,-1.0,-1.0,-1.0,-1.0,-1.0


In [49]:
df_pre_2

,day1,day2,day3,day4,day5
50,1.0,0.0,1.0,0.0,1.0
51,1.0,1.0,1.0,1.0,1.0
52,0.0,0.0,1.0,1.0,1.0
53,0.0,1.0,1.0,1.0,1.0
54,0.0,1.0,1.0,0.0,1.0
55,1.0,0.0,0.0,1.0,1.0
56,0.0,1.0,0.0,0.0,1.0
57,1.0,0.0,0.0,1.0,1.0
58,1.0,0.0,0.0,0.0,0.0
59,1.0,1.0,0.0,1.0,0.0


In [64]:
interval = 0
ETF_week_score = []

for code in ETF_list:
    ETF_price,ETF_rate,mrf = get_targetdata(ETFtable,code)
    df_list = [ETF_price]+features
    df_option = [True,True,True,True,True,True,False,True,True,True,True,True,False,True,True]
    df_name = ['Yunta50','orderbook','bond','TXF','EXF','FXF','E_F','Nikkei','SP','Russell','DJ','VIX','pcr','NAS','OI']
    print('Code:',code)
    #get day data
    
    df_total,trf,rf = get_dataframe_drop_5(df_list,df_option,ETF_rate,interval,window = 1)
    df_total2,trf2,rf2 = get_dataframe_drop_5(df_list,df_option,ETF_rate,interval,window = 2)
    df_total3,trf3,rf3 = get_dataframe_drop_5(df_list,df_option,ETF_rate,interval,window = 3)
    
    #extra
    pcr_1 = get_change_rate(features[1],1).iloc[:,features[-3].shape[1]:]
    bond_1 = get_change_rate(features[1],1).iloc[:,features[1].shape[1]:]
    orderbook_50 = get_change_rate(features[0],50).iloc[:,features[0].shape[1]:]
    orderbook_100 = get_change_rate(features[0],100).iloc[:,features[0].shape[1]:]
    
    df_total30,trf30,rf30 = get_dataframe_drop_5(df_list,df_option,ETF_rate,interval,window = 30)
    
    df_total = pd.concat([df_total,orderbook_50,orderbook_100],axis = 1).dropna(axis = 0,how = 'any')
    df_total2 = pd.concat([df_total2,df_total30,pcr_1,bond_1,orderbook_50,orderbook_100],axis = 1).dropna(axis = 0,how = 'any')
    df_total3 = pd.concat([df_total2,df_total30,pcr_1,bond_1,orderbook_50,orderbook_100],axis = 1).dropna(axis = 0,how = 'any')
    df_total3 = pd.concat([df_total3,df_total30],axis = 1).dropna(axis = 0,how = 'any')
    df_total4 = pd.concat([df_total2,df_total30,pcr_1,bond_1,orderbook_50,orderbook_100],axis = 1).dropna(axis = 0,how = 'any')
    df_total5 = pd.concat([df_total2,df_total30,pcr_1,bond_1,orderbook_50,orderbook_100],axis = 1).dropna(axis = 0,how = 'any')
    
    diff = int(df_total.shape[0] - df_total2.shape[0])
    stock_5 = [df_total.iloc[diff:].as_matrix(),df_total2.shift(1).as_matrix(),df_total3.shift(2).as_matrix(),df_total4.shift(3).as_matrix(),df_total5.shift(4).as_matrix()]
    rf_diff = rf.iloc[diff:]
    rf_5 = [rf_diff.as_matrix(),rf_diff.as_matrix(),rf_diff.as_matrix(),rf_diff.as_matrix(),rf_diff.as_matrix()]
    
    #train and test  
    pred_day = 1
    latest_day = [30,30,30,30,30]
    day = 5
    pip = Model_Selection3(models,model_grid_params,stock_5,rf_5,latest_day,pred_day,day)
    
    pip.pipline_pre_4()
    
    ETF_week_score.append(pip.week_score)
df_score_week = pd.DataFrame(ETF_week_score,columns=['week1','week2','week3','week4'])
df_score_week.index = ETF_list

Code: 50
Day = 1
--------------------Rolling Window Time = 0--------------------
Running GridSearchCV for RandomForestClassifier
{'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 50}
CV Best Score = 0.6333333333333333
Running training & testing for RandomForestClassifier.
Accuracy = 0.0
Day = 2
--------------------Rolling Window Time = 0--------------------
Running GridSearchCV for RandomForestClassifier
{'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 50}
CV Best Score = 0.6666666666666666
Running training & testing for RandomForestClassifier.
Accuracy = 0.0
Day = 3
--------------------Rolling Window Time = 0--------------------
Running GridSearchCV for RandomForestClassifier
{'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 50}
CV Best Score = 0.6333333333333333
Running training & testing for RandomForestClassifier.
Accuracy = 1.

{'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 50}
CV Best Score = 0.5333333333333333
Running training & testing for RandomForestClassifier.
Accuracy = 0.0
Day = 2
--------------------Rolling Window Time = 1--------------------
Running GridSearchCV for RandomForestClassifier
{'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 50}
CV Best Score = 0.5333333333333333
Running training & testing for RandomForestClassifier.
Accuracy = 0.0
Day = 3
--------------------Rolling Window Time = 1--------------------
Running GridSearchCV for RandomForestClassifier
{'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 20}
CV Best Score = 0.6
Running training & testing for RandomForestClassifier.
Accuracy = 0.0
Day = 4
--------------------Rolling Window Time = 1--------------------
Running GridSearchCV for RandomForestClassifier
{'max_depth': 2, 'max_

{'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 50}
CV Best Score = 0.5
Running training & testing for RandomForestClassifier.
Accuracy = 0.0
Day = 3
--------------------Rolling Window Time = 2--------------------
Running GridSearchCV for RandomForestClassifier
{'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.5333333333333333
Running training & testing for RandomForestClassifier.
Accuracy = 1.0
Day = 4
--------------------Rolling Window Time = 2--------------------
Running GridSearchCV for RandomForestClassifier
{'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.5666666666666667
Running training & testing for RandomForestClassifier.
Accuracy = 1.0
Day = 5
--------------------Rolling Window Time = 2--------------------
Running GridSearchCV for RandomForestClassifier
{'max_depth': 2, 'max_

{'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.5666666666666667
Running training & testing for RandomForestClassifier.
Accuracy = 1.0
Day = 3
--------------------Rolling Window Time = 3--------------------
Running GridSearchCV for RandomForestClassifier
{'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.7333333333333333
Running training & testing for RandomForestClassifier.
Accuracy = 0.0
Day = 4
--------------------Rolling Window Time = 3--------------------
Running GridSearchCV for RandomForestClassifier
{'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 20}
CV Best Score = 0.6666666666666666
Running training & testing for RandomForestClassifier.
Accuracy = 0.0
Day = 5
--------------------Rolling Window Time = 3--------------------
Running GridSearchCV for RandomForestClassifier
{'max_d

{'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.5
Running training & testing for RandomForestClassifier.
Accuracy = 0.0
Day = 4
--------------------Rolling Window Time = 0--------------------
Running GridSearchCV for RandomForestClassifier
{'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 50}
CV Best Score = 0.6
Running training & testing for RandomForestClassifier.
Accuracy = 1.0
Day = 5
--------------------Rolling Window Time = 0--------------------
Running GridSearchCV for RandomForestClassifier
{'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 20}
CV Best Score = 0.6
Running training & testing for RandomForestClassifier.
Accuracy = 1.0
Day = 1
--------------------Rolling Window Time = 1--------------------
Running GridSearchCV for RandomForestClassifier
{'max_depth': 2, 'max_features': None, 'min_samples_

...

{'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.6666666666666666
Running training & testing for RandomForestClassifier.
Accuracy = 1.0
Day = 3
--------------------Rolling Window Time = 2--------------------
Running GridSearchCV for RandomForestClassifier
{'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 20}
CV Best Score = 0.6
Running training & testing for RandomForestClassifier.
Accuracy = 1.0
Day = 4
--------------------Rolling Window Time = 2--------------------
Running GridSearchCV for RandomForestClassifier
{'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 20}
CV Best Score = 0.6
Running training & testing for RandomForestClassifier.
Accuracy = 1.0
Day = 5
--------------------Rolling Window Time = 2--------------------
Running GridSearchCV for RandomForestClassifier
{'max_depth': 2, 'max_features': None

{'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 50}
CV Best Score = 0.5666666666666667
Running training & testing for RandomForestClassifier.
Accuracy = 1.0
Day = 4
--------------------Rolling Window Time = 3--------------------
Running GridSearchCV for RandomForestClassifier
{'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 50}
CV Best Score = 0.5666666666666667
Running training & testing for RandomForestClassifier.
Accuracy = 1.0
Day = 5
--------------------Rolling Window Time = 3--------------------
Running GridSearchCV for RandomForestClassifier
{'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 20}
CV Best Score = 0.6
Running training & testing for RandomForestClassifier.
Accuracy = 0.0
Code: 6208
Day = 1
--------------------Rolling Window Time = 0--------------------
Running GridSearchCV for RandomForestClassifier
{'max_depth

{'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.5333333333333333
Running training & testing for RandomForestClassifier.
Accuracy = 1.0
Day = 5
--------------------Rolling Window Time = 0--------------------
Running GridSearchCV for RandomForestClassifier
{'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 50}
CV Best Score = 0.6
Running training & testing for RandomForestClassifier.
Accuracy = 0.0
Day = 1
--------------------Rolling Window Time = 1--------------------
Running GridSearchCV for RandomForestClassifier
{'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 20}
CV Best Score = 0.6
Running training & testing for RandomForestClassifier.
Accuracy = 0.0
Day = 2
--------------------Rolling Window Time = 1--------------------
Running GridSearchCV for RandomForestClassifier
{'max_depth': 2, 'max_features': None

{'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 50}
CV Best Score = 0.5666666666666667
Running training & testing for RandomForestClassifier.
Accuracy = 0.0
Day = 1
--------------------Rolling Window Time = 2--------------------
Running GridSearchCV for RandomForestClassifier
{'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 50}
CV Best Score = 0.6333333333333333
Running training & testing for RandomForestClassifier.
Accuracy = 0.0
Day = 2
--------------------Rolling Window Time = 2--------------------
Running GridSearchCV for RandomForestClassifier
{'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 20}
CV Best Score = 0.6333333333333333
Running training & testing for RandomForestClassifier.
Accuracy = 1.0
Day = 3
--------------------Rolling Window Time = 2--------------------
Running GridSearchCV for RandomForestClassifier
{'max_d

{'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.5333333333333333
Running training & testing for RandomForestClassifier.
Accuracy = 1.0
Day = 2
--------------------Rolling Window Time = 3--------------------
Running GridSearchCV for RandomForestClassifier
{'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 10}
CV Best Score = 0.5666666666666667
Running training & testing for RandomForestClassifier.
Accuracy = 1.0
Day = 3
--------------------Rolling Window Time = 3--------------------
Running GridSearchCV for RandomForestClassifier
{'max_depth': 2, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 50}
CV Best Score = 0.6333333333333333
Running training & testing for RandomForestClassifier.
Accuracy = 0.0
Day = 4
--------------------Rolling Window Time = 3--------------------
Running GridSearchCV for RandomForestClassifier
{'max_d

In [65]:
rf_score = np.array([df_score_week['week1'].sum(),df_score_week['week2'].sum(),df_score_week['week3'].sum(),df_score_week['week4'].sum()])
rf_score

array([5.4  , 3.025, 5.225, 3.15 ])

In [50]:
reg_score = np.array([8.935414422645772,8.936320370006342,8.940148717637001,8.948457738769086])

In [51]:
#30 10 30 10 10
rf_score+reg_score 

array([14.66041442, 10.56132037, 12.46514872, 13.67345774])

In [55]:
# 10 10 10 10 10
rf_score+reg_score 

array([14.66041442, 10.36132037, 12.31514872, 13.67345774])

In [59]:
# 10 10 10 10 10 original
rf_score+reg_score 

array([14.76041442, 10.31132037, 12.26514872, 13.82345774])

In [63]:
# 30 30 30 30 30 original
rf_score+reg_score 

array([13.71041442, 11.66132037, 14.59014872, 12.29845774])

In [66]:
# 30 30 30 30 30 original model check
rf_score+reg_score 

array([14.33541442, 11.96132037, 14.16514872, 12.09845774])

In [ ]:
rf_score+reg_score 

In [52]:
df_score_week

,week1,week2,week3,week4
50,0.275,0.050,0.000,0.275
51,0.400,0.050,0.125,0.450
52,0.200,0.000,0.225,0.375
53,0.175,0.200,0.150,0.050
54,0.275,0.100,0.175,0.350
55,0.275,0.075,0.325,0.250
56,0.425,0.100,0.200,0.200
57,0.325,0.000,0.425,0.250
58,0.275,0.050,0.150,0.350
59,0.275,0.200,0.275,0.225
